Author: Amiri Hayes \
Date Updated: 7/15/25 \
Title: ViewLLM

Starter code !

In [ ]:
# package installs if necessary
!pip install torch spacy nltk tqdm transformers datasets scikit-learn openai
!python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
# IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import spacy
nlp = spacy.load("en_core_web_sm")
import nltk
nltk.download('punkt_tab')
import os
import csv
import json
import requests
from dotenv import load_dotenv, find_dotenv
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
from typing import Optional, Tuple, Callable
import matplotlib.patches as mpatches
from matplotlib.colors import PowerNorm, ListedColormap
from sklearn.linear_model import LinearRegression
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from transformers import PreTrainedTokenizerBase, PreTrainedModel
from openai import OpenAI
load_dotenv(find_dotenv())
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"),)
print("Imports completed successfully.")

<div style="text-align: center; font-size: 2em;">
  <hr>
    Data & Pattern Initialization Functions
  <hr>
</div>

In [ ]:
# IMPORT THE PROGRAM DATABASE:

from programs import *

In [ ]:
# GENERATE & VISUALIZE ATTENTION PATTERN SCORES FROM PROGRAMS

def js_divergence(p: np.ndarray, q: np.ndarray) -> float:
    p = np.clip(p, 1e-12, 1.0)
    q = np.clip(q, 1e-12, 1.0)
    p /= p.sum()
    q /= q.sum()
    m = 0.5 * (p + q)
    return 0.5 * (np.sum(p * np.log(p / m)) + np.sum(q * np.log(q / m)))

def score_prediction(torch_model: PreTrainedModel, torch_tokenizer: PreTrainedTokenizerBase, head_loc: Tuple[int, int], pattern: Callable, sentence_1: str, sentence_2: Optional[str] = None, distance="jsd", output=False):
    layer, head = head_loc
    tokens = torch_tokenizer(sentence_1, return_tensors="pt")

    if torch_model.config.is_encoder_decoder:
        decoder_input_ids = tokens["input_ids"]
        outputs = torch_model(input_ids=tokens["input_ids"], decoder_input_ids=decoder_input_ids, output_attentions=True)
        att = outputs.encoder_attentions[layer][0, head].detach().numpy()
        name, pred_att = pattern(sentence_1, torch_tokenizer)

    else:
        if sentence_2 and pattern.__name__ == "chainofthought_pattern":
            name = "Chain of Thought Pattern"
            tokens_2 = torch_tokenizer(sentence_2, return_tensors="pt")

            att = torch_model(**tokens_2, output_attentions=True).attentions[layer][0, head].detach().numpy()
            pred_att = torch_model(**tokens, output_attentions=True).attentions[layer][0, head].detach().numpy()

            if output: print("RUNNING FIRST WITH NO HINT")
            question, answer, vector_att = chainofthought_pattern(sentence_1, torch_tokenizer, pred_att, hint=False)
            if output: print("RUNNING AFTER WITH A HINT")
            question, answer, vector_pred_att = chainofthought_pattern(sentence_2, torch_tokenizer, att, hint=True)

            att, pred_att = vector_att.copy(), vector_pred_att.copy()
        else:
            att = torch_model(**tokens, output_attentions=True).attentions[layer][0, head].detach().numpy()
            if pattern.__name__ == "linear_fit":
                name, pred_att = pattern(sentence_1, torch_tokenizer, idx=0)
            else: name, pred_att = pattern(sentence_1, torch_tokenizer)

    if distance == "raw":
        score = np.abs(att - pred_att).sum()
    elif distance == "jsd":
        jensonshannon_distances = []
        for row_att, row_out in zip(att, pred_att):
            jensonshannon_distances.append(np.sqrt(js_divergence(row_att, row_out)))
        score = np.mean(jensonshannon_distances)

    if pattern.__name__ == "chainofthought_pattern":
        score = np.sqrt(js_divergence(att, pred_att))

    if output == "cot":
        colors = "inferno"
        fig, axes = plt.subplots(1, 2, figsize=(12, 9))
        axes[0].plot(att, color=plt.get_cmap(colors)(0.6))
        axes[0].set_title("Actual Head Attention")
        axes[1].plot(pred_att, color=plt.get_cmap(colors)(0.9))
        axes[1].set_title("Optimal Head Attention for Pattern")
        bound_axes = False
        for i in range(2):
            axes[i].set_xlabel("Token Index")
            axes[i].set_ylabel("Attention Weight")
            axes[i].grid(True)
            if bound_axes:
                axes[i].set_ylim(0, 1)
                axes[i].set_xlim(0, len(att) - 1)
        underlined_name_unicode = "".join([char + '\u0332' for char in name])
        question_chart = question.replace(".", ".\n")
        plt.suptitle(f"Results: {underlined_name_unicode} @ L{layer},H{head} | Raw Score = {score:.2f}\n\nQuestion: \"{question_chart}\n\nAnswer: \"{answer}\"", fontsize=16)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()
    
    toks = torch_tokenizer([sentence_1], return_tensors="pt")
    token_ids = toks["input_ids"][0]
    tokens = torch_tokenizer.convert_ids_to_tokens(token_ids)

    if output == True:
        colors="Greens"
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        im1 = axes[0].imshow(att, cmap=colors, aspect='auto')
        axes[0].set_title("Actual Head Attention")
        fig.colorbar(im1, ax=axes[0])
        im2 = axes[1].imshow(pred_att, cmap=colors, aspect='auto')
        axes[1].set_title("Optimal Head Attention for Pattern")
        fig.colorbar(im2, ax=axes[1])
        for i in range(2):
            axes[i].set_xticks(range(len(tokens)))
            axes[i].set_yticks(range(len(tokens)))
            # get rid of the weird special characters in each token in tokens
            for token in tokens:
                if token.startswith("Ġ"):
                    tokens[tokens.index(token)] = token[1:]
            axes[i].set_xticklabels(tokens, rotation=90)
            axes[i].set_yticklabels(tokens)
        underlined_name_unicode = "".join([char + '\u0332' for char in name])
        plt.suptitle(f"Results: {underlined_name_unicode} @ L{layer},H{head} | Raw Score = {score:.2f}\n\nSentence: \"{sentence_1}\"", fontsize=16)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

    elif output == "optimal":
        colors = "Oranges"
        fig, ax = plt.subplots(1, 1, figsize=(6, 5))
        im2 = ax.imshow(pred_att, cmap=colors, aspect='auto')
        ax.set_axis_off()
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

    elif output == "actual":
        colors = "Reds"
        fig, ax = plt.subplots(1, 1, figsize=(6, 5))
        im2 = ax.imshow(att, cmap=colors, aspect='auto')
        ax.set_title("Example Head Attention for Pattern")
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

    return score

In [ ]:
# DOWNLOAD ADDITIONAL TEXT DATA (1):

file = 'data/small_text.csv'
df = pd.read_csv(file)

sentences = []
for paragraph in df['text']:
    sentences.extend(sent_tokenize(paragraph))

sentences = sentences[:10_000]
print("Sentences from Tiny Stories Dataset:")
for sentence in sentences[20:30]:
  print(f"\t{sentence}")

df_json = pd.read_json('data/generic_sentences.json')
generic_sentences = df_json[0].tolist()
print("\nGeneric Sentences:")
for sentence in generic_sentences[:10]:
  print(f"\t{sentence}")

In [ ]:
# DOWNLOAD ADDITIONAL TEXT DATA (2):

math_data = pd.read_json('data/math_problems_results.jsonl', lines=True)

filtered_results = math_data[
    (math_data['consistency'] == "False") &
    (math_data['evaluated_answer_nohint'] != "DNF: llm did not finish") &
    (math_data['evaluated_answer_hint'] != "DNF: llm did not finish")
]

answers_nohint = filtered_results['answer_nohint'].tolist()
answers_hint = filtered_results['answer_hint'].tolist()
prefix = "system\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nuser\n\n"

prompts = []
for s1, s2 in zip(answers_nohint, answers_hint):
    if s1.startswith(prefix): s1 = s1[len(prefix):]
    if s2.startswith(prefix): s2 = s2[len(prefix):]

    i_suffix_s1 = s1.find("assistant")
    if i_suffix_s1 != -1: s1 = s1[:i_suffix_s1].strip()

    i_suffix_s2 = s2.find("assistant")
    if i_suffix_s2 != -1: s2 = s2[:i_suffix_s2].strip()

    if s1 and s2: prompts.append((s1, s2))

print(len(prompts), "relevant prompts loaded from math problems dataset.")

<div style="text-align: center; font-size: 2em;">
  <hr>
  Analysis Helper Functions
  <hr>
</div>

In [ ]:
# ANALYZE PATTERN AT LAYER AND HEAD

sentence = "The quick brown fox jumps over the lazy dog. That he does."
sentence = "Hi. How are you? I'm fine! Thanks. Bye, see you tomorrow."
# sentence = "I like apples and I like bananas. I like apples more though."

models = ["bert-base-uncased", "openai-community/gpt2", "google-t5/t5-small", "Meta-Llama-3.1-8B-Instruct"]

i = 0
name = models[i]

if i in [0,1,2]: # small models ( <400M Params )
    tokenizer = AutoTokenizer.from_pretrained(name)
    model = AutoModel.from_pretrained(name, output_attentions=True)
    model.eval()

elif i == 3: # big model & cot ( 8B Params, Load from Compute Node )
    tokenizer = AutoTokenizer.from_pretrained("/raid/lingo/models/Meta-Llama-3.1-8B-Instruct/")
    model = AutoModelForCausalLM.from_pretrained("/raid/lingo/models/Meta-Llama-3.1-8B-Instruct/")
    model.eval()

    prompt_num = 0
    sentence = prompts[prompt_num][0]  # Use the prompt's first sentence (no hint)
    sentence_with_hint = prompts[prompt_num][1]  # Use prompt's second sentence (hint)

layer, head = 1,1
# sentence = "he said no, he did not, he will not, he feels weirdly good about it"_
score_prediction(model, tokenizer, (layer, head), next_attention, sentence, distance="jsd", output=True)

In [ ]:
# ANALYZE ALL HEADS FOR A PATTERN (1 SENTENCE)

def visualize_full_model(sentence: str, torch_model: PreTrainedModel, torch_tokenizer: PreTrainedTokenizerBase, pattern: Callable, title: str, bias_towards_best: float = 0.9, sentence_2: Optional[str] = None) -> Tuple[np.ndarray, list]:
    num_layers = model.config.num_hidden_layers
    num_heads = model.config.num_attention_heads
    model_viz = np.zeros((num_layers, num_heads))

    for i in range(num_layers):
        for j in range(num_heads):
            score = score_prediction(torch_model, torch_tokenizer, (i, j), pattern, sentence, distance="jsd", output=False)
            model_viz[i, j] = score
    print(f"Best Score: {min(map(min, model_viz)):.2f}")

    plt.figure(figsize=(12, 6))
    norm = PowerNorm(gamma=1-bias_towards_best, vmin=model_viz.min(), vmax=model_viz.max())
    plt.imshow(model_viz, cmap='Greens_r', aspect='auto', norm=norm)
    plt.colorbar()
    plt.title(title, fontsize=16)
    plt.xlabel("Attention Heads")
    plt.ylabel("Transformer Layers")
    plt.xticks(ticks=np.arange(num_heads), labels=[f'H{h}' for h in range(num_heads)])
    plt.yticks(ticks=np.arange(num_layers), labels=[f'L{l}' for l in range(num_layers)])
    plt.tight_layout()
    plt.show()

    top_three = list(zip(*np.unravel_index(np.argsort(model_viz, axis=None)[:3], model_viz.shape)))
    for tuple_val in top_three:
        layer, head = tuple_val
        score = model_viz[layer, head]
        print(f"Layer {layer}, Head {head} - Score: {score:.2f}")
    return model_viz, top_three

model_name = f"Model = {model.config.architectures[0]}\n"
sentence = "Hi. How are you? I'm fine! Thanks. Bye, see you tomorrow."
pattern_name = "Pattern Name Here"
underlined_name_unicode = "".join([char + '\u0332' for char in pattern_name])
visualize_full_model(sentence, model, tokenizer, adverbial_modulation, title=f"Top Heads: {underlined_name_unicode} | {model_name}\nSentence: \"{sentence}\"\n", bias_towards_best=0.7)

In [ ]:
# ANALYZE ALL HEADS FOR A PATTERN (MULTIPLE SENTENCES)

def visualize_full_model(sentences: list[str], torch_model: PreTrainedModel, torch_tokenizer: PreTrainedTokenizerBase, pattern: Callable, title: str, bias_towards_best: float = 0.9) -> Tuple[np.ndarray, list]:
    num_layers = torch_model.config.num_hidden_layers
    num_heads = torch_model.config.num_attention_heads

    header = ["Layer", "Head", "Score"]
    csv_file_name = "scores.csv"
    file_exists = os.path.exists(csv_file_name)
    with open(csv_file_name, 'a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(header)
    
        average_score = np.zeros((num_layers, num_heads))
        for sentence in sentences:
            sentence_1 = sentence[0]  # first sentence (no hint)
            sentence_2 = sentence[1]  # second sentence (hint)
            model_score = np.zeros((num_layers, num_heads))
            for i in range(num_layers):
                for j in range(num_heads):
                    score = score_prediction(model, tokenizer, (i, j), chainofthought_pattern, sentence_1, sentence_2, distance="jsd", output=False)
                    writer.writerow([i, j, f"{score:.2f}"])
                    print(f"Layer {i}, Head {j} - Score: {score:.2f}")
                    model_score[i, j] = score
            average_score += model_score
        average_score /= len(sentences)

    plt.figure(figsize=(12, 6))
    norm = PowerNorm(gamma=1-bias_towards_best, vmin=average_score.min(), vmax=average_score.max())
    plt.imshow(average_score, cmap='Reds', aspect='auto', norm=norm)
    plt.colorbar()
    plt.title(title, fontsize=16)
    plt.xlabel("Attention Heads")
    plt.ylabel("Transformer Layers")
    plt.xticks(ticks=np.arange(num_heads), labels=[f'H{h}' for h in range(num_heads)])
    plt.yticks(ticks=np.arange(num_layers), labels=[f'L{l}' for l in range(num_layers)])
    plt.tight_layout()
    plt.show()

    if pattern.__name__ == "chainofthought_pattern": 
        top_three = list(zip(*np.unravel_index(np.argsort(average_score.ravel())[::-1][:3], average_score.shape))) # highest scores
    else:
        top_three = list(zip(*np.unravel_index(np.argsort(average_score, axis=None)[3:], average_score.shape))) # lowest scores
        top_three = np.sort(average_score)

    for tuple_val in top_three:
            layer, head = tuple_val
            score = average_score[layer, head]
            print(f"Layer {layer}, Head {head} - Score: {score:.2f}")
    return average_score, top_three

model_name = f"Model = {model.config.architectures[0]}\n"
sentences_zipped = list(zip(answers_nohint[:5], answers_hint[:5]))
visualize_full_model(sentences_zipped, model, tokenizer, relative_position_attention, title="Top Heads: Chain-of_Thought Evaluation [AVERAGE]\n"+model_name, bias_towards_best=0.7)

In [ ]:
# FILTER SENTENCES TO TARGET SPECIFIC PATTERNS

def filterer(sentences: list[str], length_matters: bool=False, punctuation_matters: bool=False, duplicates: bool=False) -> list[str]:
    filtered_sentences = []
    punctuation_set = set('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
    for sentence in sentences:
        passes_all_active_filters = True
        if length_matters:
            if len(sentence.split()) < 5:
                passes_all_active_filters = False
                continue
        if punctuation_matters:
            punctuation_count = sum(1 for char in sentence if char in punctuation_set)
            if punctuation_count < 3:
                passes_all_active_filters = False
                continue
        if duplicates:
            set_l = len(set(sentence.split()))
            sent_1 = len(sentence.split())
            if set_l >= sent_1 - 3:
                passes_all_active_filters = False
                continue
        if passes_all_active_filters: filtered_sentences.append(sentence)
    return filtered_sentences

filtered_sentences = filterer(generic_sentences, length_matters = True)

In [ ]:
# ANALYZE HEAD PATTERN ON SELECTED SENTENCES

def visualize_highest_head(sentences: list[str], top_n:  int, torch_model: PreTrainedModel, torch_tokenizer: PreTrainedTokenizerBase, head_loc: tuple[int, int], pattern: Callable):
    layer, head = head_loc
    scores = []

    for sentence in tqdm(sentences):
        score = score_prediction(torch_model, torch_tokenizer, (layer, head), pattern, sentence, distance="jsd", output=False)
        scores.append(score)

    indexed_scores = list(enumerate(scores))
    sorted_scores = sorted(indexed_scores, key= lambda x: x[1])
    top_scores = sorted_scores[:top_n]

    for idx, score in top_scores:
        print(f"Sentence #{idx} Score: {score}")
        sentence = sentences[idx]
        score_prediction(torch_model, torch_tokenizer, (layer, head), pattern, sentence, output=True)
    return 0

layer, head = 7, 8
visualize_highest_head(filtered_sentences, 3, model, tokenizer, (layer, head), punctuation_attention)

In [ ]:
# DETERMINE BEST FIT PATTERNS FOR ALL HEADS

def classify_whole_model(sentences: list[str], torch_model: PreTrainedModel, torch_tokenizer: PreTrainedTokenizerBase, patterns: list[Callable]) -> dict[Tuple[int, int], Tuple[str, float]]:
    num_layers = torch_model.config.num_hidden_layers
    num_heads = torch_model.config.num_attention_heads
    activations = {}  # key: (i, j), value: (pattern_name, score)
    header = ["i", "j", "Pattern", "Score"]
    
    csv_file_name = "data/best_fit_refinement_gpt2_new.csv"
    with open(csv_file_name, 'a', newline='') as file:
        writer = csv.writer(file)
        file_exists = os.path.exists(csv_file_name)
        if not file_exists:
            writer.writerow(header)

        for pattern in patterns:
            try:
                print(f"\nCurrently Analyzing pattern: {pattern.__name__}")
                all_scores = []
                for idx, sentence in enumerate(sentences):
                    if idx % 20 == 0: print(f"\tProcessing sentence {idx}/{len(sentences)}")
                    for i in range(num_layers):
                        for j in range(num_heads):
                            score = score_prediction(torch_model, torch_tokenizer, (i, j), pattern, sentence, distance="jsd", output=False)
                            if score < 0.55: print(f"sentence #{idx}|", i, j, score)
                            all_scores.append(score)

                average_scores = np.array(all_scores).reshape(len(sentences), num_layers * num_heads).mean(axis=0)
                head_performance = average_scores.reshape(num_layers, num_heads)
                print(head_performance)

                ix, jx = np.where(head_performance < 0.45)
                pairs = list(zip(ix, jx))

                for (ix, jx) in pairs:
                    print(ix, jx, head_performance[ix, jx])
                    # if key not in activations or pattern_score < activations[key][1]:
                    #     activations[key] = (pattern.__name__, pattern_score)
                    writer.writerow([ix, jx, pattern.__name__, head_performance[ix, jx]])

            except Exception as e:
                print(f"Error processing pattern {pattern.__name__}: {e}")
                continue


            # print(len(avg_scores))
            # pattern_score = np.mean(avg_scores)
            # print(i, j, pattern_score)

            # if pattern_score > 0.5: continue
            # key = (i, j)

            # if key not in activations or pattern_score < activations[key][1]:
            #     activations[key] = (pattern.__name__, pattern_score)
            # writer.writerow([i, j, pattern.__name__, pattern_score])

    return activations

model = AutoModel.from_pretrained("openai-community/gpt2", output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")

# patterns = [next_attention, previous_attention, same_attention, punctuation_attention, repeated_attention, pos_alignment, dependencies]
# patterns = [next_attention, previous_attention, same_attention, punctuation_attention, last_token_attention, repeated_attention, uniform_attention, cls_attention, eos_attention, special_token_attention, pos_alignment, dependencies]

# model = AutoModel.from_pretrained("bert-base-uncased", output_attentions=True)
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

activations = classify_whole_model(generic_sentences[:5], model, tokenizer, patterns)
print(activations)

In [ ]:

# NEVERMIND, loop through all heads find the corresponsing programs from llm_code in automation_results_bert and bert2 and save to best_fit_refinement_bert.csv if score < 0.4

num_layers = model.config.num_hidden_layers
num_heads = model.config.num_attention_heads

for layer in range(num_layers):
    for head in range(num_heads):
        # print(f"Layer {layer}, Head {head}")
        # look for layer, head in automation_results_bert/scores (e.g. C:\Users\amkah\OneDrive\Documents\GitHub\LLM-Interpretability\automation_results_bert\scores\layer0_head0_score.txt)
        with open(f"automation_results_gpt2/scores/layer{layer}_head{head}_score.txt", 'r') as f:
            score_1 = float(f.read().strip())

        if score_1 < 0.25:
            # copy the name of the corresponding program from automation_results_bert/llm_code (e.g. C:\Users\amkah\OneDrive\Documents\GitHub\LLM-Interpretability\automation_results_bert\llm_code\layer0_head0_code.py)
            with open(f"automation_results_bert/llm_code/layer{layer}_head{head}_code.py", 'r') as f:
                # program name is after "def " and before "(", isn't necessarily the first line, read all lines and join
                all_lines = f.readlines()

                # strip all_lines of stuff before def and after ( 
                program_name = " ".join([line.strip().split(" ")[1] for line in all_lines if line.startswith("def ")])
                program_name = program_name.split("(")[0]

            # save to best_fit_refinement_bert.csv
            # with open('best_fit_refinement_bert.csv', 'a') as f:
            #     f.write(f"{layer},{head},{program_name},{score_1}\n")
            print(f"{layer},{head},{program_name},{score_1}")

            # for som reason the append isn't working


In [ ]:
from programs import *
import importlib.util
import types
patterns = [next_attention, previous_attention, same_attention, last_token_attention, repeated_attention, uniform_attention, cls_attention, eos_attention, special_token_attention, pos_alignment, dependencies]

master_list_dir = "automation_refinement_bert/master_list"
for filename in os.listdir(master_list_dir):
    if filename.endswith(".py"):
        module_name = filename[:-3]
        if module_name not in [p.__name__ for p in patterns]:
            try:
                program_path = os.path.join(master_list_dir, filename)
                spec = importlib.util.spec_from_file_location("loaded_program", program_path)
                module = importlib.util.module_from_spec(spec)
                module.__dict__['np'] = np
                # module.__dict__['torch'] = torch
                module.__dict__['spacy'] = spacy
                module.__dict__['nlp'] = nlp
                module.__dict__['PreTrainedTokenizerBase'] = PreTrainedTokenizerBase
                module.__dict__['Optional'] = Optional
                module.__dict__['Tuple'] = Tuple
                module.__dict__['Callable'] = Callable
                spec.loader.exec_module(module)
            except Exception as e:
                print(f"Program loading failed: {str(e)}")

            for attr_name in dir(module):
                attr = getattr(module, attr_name)
                if isinstance(attr, types.FunctionType):
                    program_to_test = attr
                    break

            patterns.append(program_to_test)

master_list_dir = "automation_refinement/master_list"
for filename in os.listdir(master_list_dir):
    if filename.endswith(".py"):
        module_name = filename[:-3]
        if module_name not in [p.__name__ for p in patterns]:
            try:
                program_path = os.path.join(master_list_dir, filename)
                spec = importlib.util.spec_from_file_location("loaded_program", program_path)
                module = importlib.util.module_from_spec(spec)
                module.__dict__['np'] = np
                # module.__dict__['torch'] = torch
                module.__dict__['spacy'] = spacy
                module.__dict__['nlp'] = nlp
                module.__dict__['PreTrainedTokenizerBase'] = PreTrainedTokenizerBase
                module.__dict__['Optional'] = Optional
                module.__dict__['Tuple'] = Tuple
                module.__dict__['Callable'] = Callable
                spec.loader.exec_module(module)
            except Exception as e:
                print(f"Program loading failed: {str(e)}")

            for attr_name in dir(module):
                attr = getattr(module, attr_name)
                if isinstance(attr, types.FunctionType):
                    program_to_test = attr
                    break

            patterns.append(program_to_test)

print(f"Total patterns to analyze: {len(patterns)}")
for pattern in patterns: print(f"\tPattern: {pattern.__name__}")

In [ ]:
# DETERMINE BEST FIT PATTERNS FOR ALL HEADS (CONTINUED)

model = AutoModel.from_pretrained("google-t5/t5-small", output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")

torch_model = model
torch_tokenizer = tokenizer

num_layers = torch_model.config.num_hidden_layers
num_heads = torch_model.config.num_attention_heads
activations = {}  # key: (i, j), value: (pattern_name, score)

from programs import *
patterns = [next_attention, previous_attention, same_attention, last_token_attention, repeated_attention, uniform_attention, cls_attention, eos_attention, special_token_attention, pos_alignment, dependencies]

master_list_dir = "automation_refinement_gpt2/master_list"
for filename in os.listdir(master_list_dir):
    if filename.endswith(".py"):
        module_name = filename[:-3]
        if module_name not in [p.__name__ for p in patterns]:
            try:
                program_path = os.path.join(master_list_dir, filename)
                spec = importlib.util.spec_from_file_location("loaded_program", program_path)
                module = importlib.util.module_from_spec(spec)
                module.__dict__['np'] = np
                # module.__dict__['torch'] = torch
                module.__dict__['spacy'] = spacy
                module.__dict__['nlp'] = nlp
                module.__dict__['PreTrainedTokenizerBase'] = PreTrainedTokenizerBase
                module.__dict__['Optional'] = Optional
                module.__dict__['Tuple'] = Tuple
                module.__dict__['Callable'] = Callable
                spec.loader.exec_module(module)
            except Exception as e:
                print(f"Program loading failed: {str(e)}")

            for attr_name in dir(module):
                attr = getattr(module, attr_name)
                if isinstance(attr, types.FunctionType):
                    program_to_test = attr
                    break

            patterns.append(program_to_test)

print(f"Total patterns to analyze: {len(patterns)}")
for pattern in patterns: print(f"\tPattern: {pattern.__name__}")

short = sentences[:8]
csv_file_name = "data/best_fit_refinement.csv"
with open(csv_file_name, 'a', newline='') as file:
    writer = csv.writer(file)
    for pattern in patterns:
        print(f"\nCurrently Analyzing pattern: {pattern.__name__}")
        avg_score = []
        for idx, sentence in enumerate(short):
            print(f"\tProcessing sentence {idx}/{len(short)}")
            for i in range(num_layers):
                for j in range(num_heads):
                    if i != 3 or j != 9: continue
                    score = score_prediction(torch_model, torch_tokenizer, (i, j), pattern, sentence, distance="jsd", output=False)
                    if score < 0.55:
                        avg_score.append((idx, pattern.__name__, i, j, score))
        
        score_dict = {}
        for idx, pattern_name, i, j, score in avg_score:
            score_dict.setdefault((i, j), []).append((pattern_name, score))
        for (i, j), values in score_dict.items():
            scores = [score for _, score in values]
            avg_score_val = sum(scores) / len(scores)
            pattern_name = values[0][0]
            activations[(i, j)] = (pattern_name, avg_score_val)
            print(f"Layer {i}, Head {j} - Score: {avg_score_val:.2f}")
            writer.writerow([i, j, pattern.__name__, avg_score_val])

In [ ]:
# ANALYZE EFFECT OF LINEAR WEIGHTS ON ATTENTION ACTIVATION ACCURACY

def generate_dataset(patterns: list[Callable], model: PreTrainedModel, tokenizer: PreTrainedTokenizerBase, sentences: list[str], layer_head: tuple[int, int]):
    layer, head = layer_head
    X_data, y_data = [], []
    print("Generating dataset for Layer", layer, ", Head", head)

    for sentence in tqdm(sentences):
        inputs = tokenizer(sentence, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs, output_attentions=True)
            attn = outputs.attentions[layer][0, head]
        X_i_list = []
        for pattern in patterns:
            _, X_i = pattern(sentence, tokenizer)
            X_i = torch.tensor(X_i, dtype=torch.float32)
            X_i_list.append(X_i)
        X_data.append(X_i_list)
        y_data.append(attn)

    torch.save({'X': X_data, 'y': y_data}, "data/attention_dataset.pt")
    print("Dataset generated and saved to 'data/attention_dataset.pt'.")

def train_linearregression() -> pd.DataFrame:
    data = torch.load("data/attention_dataset.pt")
    X, y = data['X'], data['y']
    X, y = data['X'], data['y']
    output = []

    for i, (xb, yb) in enumerate(zip(X, y)):
        xb = torch.stack(xb)
        X_flat = (xb.reshape(len(xb), -1).T).numpy()
        y_flat = yb.flatten().numpy()
        reg = LinearRegression().fit(X_flat, y_flat)
        if i % 100 == 0: print(f"Sentence #{i} - Coeffs: {[float(f"{coef:.2f}") for coef in reg.coef_]}, Intercept: {reg.intercept_:.2f}")
        output.append([reg.coef_.tolist(), float(reg.intercept_)])

    output = pd.DataFrame(output, columns=["Coefficients", "Intercept"]).to_csv("data/linear_regression_results.csv", index=False)
    return output

head_loc = (3, 9)
patterns = [next_attention, previous_attention, same_attention, punctuation_attention, repeated_attention, pos_alignment, dependencies]
generate_dataset(patterns, model, tokenizer, sentences, head_loc)
output = train_linearregression()

In [ ]:
# VISUALIZE BEST FIT PATTERNS ACROSS LAYERS AND HEADS

# model = AutoModel.from_pretrained("roberta-base", output_attentions=True)
# tokenizer = AutoTokenizer.from_pretrained("roberta-base")

model = AutoModel.from_pretrained("openai-community/gpt2", output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")

# model = AutoModel.from_pretrained("bert-base-uncased", output_attentions=True)
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

threshold = 0.45

df = pd.read_csv('data/best_fit_refinement_gpt2_new.csv')
# ignore columns where scores >= threshold
df = df[df['Score'] < threshold]
num_layers = model.config.num_hidden_layers
num_heads = model.config.num_attention_heads
mat = np.zeros((num_layers, num_heads), dtype=object)
for r in range(num_layers):
    for c in range(num_heads):
        mat[r, c] = []

for (i, j), group in df.groupby(['i', 'j']):
    sorted_group = group.sort_values(by='Score', ascending=False)
    mat[i, j] = [(row['Pattern'], row['Score']) for idx, row in sorted_group.iterrows()]

unique_patterns = df['Pattern'].unique()
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap, to_rgb

colors_40 = plt.cm.get_cmap('tab20').colors + plt.cm.get_cmap('tab20b').colors
custom_cmap_40 = ListedColormap(colors_40, name='tab40')
cmap_patterns = plt.get_cmap(custom_cmap_40, len(unique_patterns))

pattern_colors = {pattern: cmap_patterns(i) for i, pattern in enumerate(unique_patterns)}
white_color = (1, 1, 1, 1)
plotting_matrix_rgb = np.zeros((num_layers, num_heads, 3))

for r in range(num_layers):
    for c in range(num_heads):
        cell_data = mat[r, c]
        
        if not cell_data:
            plotting_matrix_rgb[r, c] = white_color[:3]
        elif len(cell_data) == 1: 
            pattern_name = cell_data[0][0]
            plotting_matrix_rgb[r, c] = pattern_colors[pattern_name][:3]
        else:
            top_pattern_1 = cell_data[0][0]
            top_pattern_2 = cell_data[1][0]
            color_1 = pattern_colors[top_pattern_1][:3]
            color_2 = pattern_colors[top_pattern_2][:3]
            plotting_matrix_rgb[r, c] = color_1 

custom_draw_mask = np.zeros((num_layers, num_heads), dtype=bool)
for r in range(num_layers):
    for c in range(num_heads):
        if len(mat[r, c]) > 1:
            custom_draw_mask[r, c] = True

fig, ax = plt.subplots(figsize=(9, 18))
img = ax.imshow(plotting_matrix_rgb, origin='lower', extent=[-0.5, num_heads - 0.5, -0.5, num_layers - 0.5])

for r in range(num_layers):
    for c in range(num_heads):
        if custom_draw_mask[r, c]:
            cell_data = mat[r, c]
            top_pattern_1 = cell_data[0][0]
            top_pattern_2 = cell_data[1][0]
            
            color_1 = pattern_colors[top_pattern_1]
            color_2 = pattern_colors[top_pattern_2]
            triangle1 = mpatches.Polygon(
                [[c - 0.5, r - 0.5], [c + 0.5, r - 0.5], [c + 0.5, r + 0.5]],
                facecolor=color_1, edgecolor='none'
            )
            ax.add_patch(triangle1)
            triangle2 = mpatches.Polygon(
                [[c - 0.5, r - 0.5], [c - 0.5, r + 0.5], [c + 0.5, r + 0.5]],
                facecolor=color_2, edgecolor='none'
            )
            ax.add_patch(triangle2)

ax.set_xticks(np.arange(num_heads))
ax.set_yticks(np.arange(num_layers))
ax.set_xticks(np.arange(-0.5, num_heads, 1), minor=True)
ax.set_yticks(np.arange(-0.5, num_layers, 1), minor=True)
ax.set_xlabel(f'{model.config.architectures[0]} - Heads', fontsize=14)
ax.set_ylabel(f'{model.config.architectures[0]} - Layers', fontsize=14)
# center title across whole figure
# ax.set_title('What is GPT doing? - Best Fit Patterns Across All Heads and Layers', fontsize=18, pad=20, loc='center')
ax.set_aspect('equal')
# ax.grid(color='black', linestyle='-', linewidth=0.5)
ax.grid(which='minor', color='black', linestyle='-', linewidth=0.5)

legend_handles = []
for pattern, color in pattern_colors.items():
    patch = mpatches.Patch(color=color, label=pattern)
    legend_handles.append(patch)

# center title
title_value_text = 'What is BERT doing? - Best Fit Patterns Across Attention Heads'
underlined_title_unicode = "".join([char + '\u0332' for char in title_value_text])
# plt.suptitle(underlined_title_unicode, fontsize=20, y=0.725, x=0.4)
# sort handles alphabetically by label
# legend_handles = sorted(legend_handles, key=lambda x: x.get_label())

white_patch = mpatches.Patch(facecolor=white_color, label='No Hypotheses Validated', edgecolor='black', linewidth=0.2)
legend_handles.append(white_patch)

name = 'Confidence'
underlined_name_unicode = "".join([char + '\u0332' for char in name])

ax.legend(
    handles=legend_handles, 
    loc='center left', 
    bbox_to_anchor=(1.05, 0.5),
    ncol=3, 
    fancybox=True, 
    shadow=True, 
    title=f"HIGH SCORING HYPOTHESES - {model.config.architectures[0]} | {underlined_name_unicode}: avg_score < {threshold}\n",
    title_fontsize=16,
    fontsize='large', # Make legend text bigger. Can use 'medium', 'x-large', 'xx-large' or a numerical value (e.g., 12)
    labelspacing=1.5, # Adjust vertical spacing between legend entries (default is 0.5)
    handlelength=2.5, # Adjust length of the color patch/line in the legend
    handletextpad=0.8, # Adjust space between the handle (color patch) and the text label
    borderpad=0.5 # Adjust padding between the legend content and its border 
) 

# plt.tight_layout()
# plt.show()

plt.tight_layout(rect=[0, 0.1, 1, 1]) # Adjust layout to make space for the legend
plt.show()

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

response = client.responses.create(
    model="gpt-4o",
    instructions="You are a coding assistant that talks like a pirate.",
    input="How do I check if a Python object is an instance of a class?",
)

print(response.output_text)

In [ ]:
# prompt pieces (sequential)
# 1 look at attention matrix and try to predict pattern
# 2. come up with five candidate hypotheses and pick the top one
# 3. explain why you picked that hypothesis
# 4. write code that generates that pattern according to the hypothesis

from automation_helper import example_one, example_two, example_three
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

layer = 3
head = 9
sentences = sentences[:10]
model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

data = {
        "layer": layer,
        "head": head,
        "model": model.config.architectures[0],
        "examples": []}

def handle_score(score):
        # convert to percentage with 0 decimal places
        return "{:.0f}".format(score * 100)

def scrape_head(att, tokens, top_k_ratio, ignore_special=True):
        seq_len = att.shape[0]
        ignore_indices = {i for i, tok in enumerate(tokens) if ignore_special and tok in ("[CLS]", "[SEP]", "[PAD]")}
        keep_indices = [i for i in range(seq_len) if i not in ignore_indices]
        att_scores = []
        for i in keep_indices:
            for j in keep_indices:
                att_scores.append((i, j, att[i, j]))
        top_k = max(1, int(len(att_scores) * top_k_ratio))
        top_att = sorted(att_scores, key=lambda x: x[2], reverse=True)[:top_k]
        top_activations = []
        for i, j, score in top_att:
            top_activations.append(f"[{str(tokens[i])}|{str(tokens[j])}:{handle_score(score)}]")
        #make top activations str and delete brackets
        top_activations_str = " ".join(top_activations).replace("[", "").replace("]", "")
        # print(top_activations_str)
        return top_activations_str

for idx, sentence in enumerate(sentences):
        inputs = tokenizer(sentence, return_tensors="pt")
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        with torch.no_grad():
            outputs = model(**inputs, output_attentions=True)
            att = outputs.attentions[layer][0, head]
        att = att.detach().cpu().numpy()
        top_activations = scrape_head(att, tokens, top_k_ratio=0.05)
        item = {f"sentence {idx}": " ".join(tokens), "sentence attention": top_activations}
        data["examples"].append(item)

prompt_1 = f"""
        You are given the following attention matrices sourced from {model.config.architectures[0]} based on {len(sentences)} sentences
        from Layer {layer}, Head {head}. Look at the attenion matrix subset below and try to predict the most fitting three hypotheses
        for the head function. Then choose what you think is the best hypothesis and explain why you picked that one. DATA: {data}
        """

prompt_2 = f"""
        Now use your explanation to write a Python function that generates an attention matrix for any given sentence according to your hypothesis  in less than 50 lines. The input to your function is def fn(sentence: str, tokenizer: PreTrainedTokenizerBase) -> Tuple[str, np.ndarray]: and 
        the output is a tuple of a string (the name of the pattern) and a numpy array (the attention matrix). Your response should be a single code block with no extra text, and must be wrapped in ```python``` and include imports.
        Think carefully before generating any code. These patterns can be simple or complex.  For uniformity, the first three lines of your function must be 'toks = tokenizer([sentence], return_tensors="pt") len_seq = len(toks.input_ids[0]) out = np.zeros((len_seq, len_seq))'.
        Make sure the token sequences from your tokenizer and spaCy (if you must use spaCy) are aligned via a dictionary if necessary, because they split text differently. Make sure you generalize your hypothesis pattern to any sentence. Functions can almost 
        always be expressed in fewer than 50 lines of code. As examples, it has been discovered one head is responsible for the complex task of dependency parsing. It's simplistic predicted pseudocode looks like: 
        {example_one}. Example 2: '''{example_two}''' Example 3: '''{example_three}'''. Always finalize your code with normalization to ensure rows sum to 1 and the output is a valid attention matrix.
        """

# ----- AGENT GENERATES HYPOTHESIS ----- #
conversation_history = [
    {"role": "system",
    "content": "You are a coding assistant well-versed in linguistics."}
]
conversation_history.append({"role": "user", "content": prompt_1})
response_1 = client.chat.completions.create(
    model="gpt-4o", messages=conversation_history, temperature = 0.5
)
assistant_response_1 = response_1.choices[0].message.content
conversation_history.append({"role": "assistant", "content": assistant_response_1})
print(f"--- Response 1 ---\n{assistant_response_1}")

# ----- AGENT WRITES PYTHON PROGRAM ----- #
prompt_2 = "Now, give me a Python function that uses a linguistic concept from your previous explanation."
conversation_history.append({"role": "user", "content": prompt_2})
response_2 = client.chat.completions.create(
    model="gpt-5",
    messages=conversation_history
)
assistant_response_2 = response_2.choices[0].message.content
print(f"\n--- Response 2 ---\n{assistant_response_2}")

# ----- SAVE AGENT RESULTS ----- #
folder = "automation_results_gpt4o"
if not os.path.exists(folder): os.makedirs(folder)
subfolders = ["prompts", "llm_code"]
for subfolder in subfolders:
    if not os.path.exists(os.path.join(folder, subfolder)):
        os.makedirs(os.path.join(folder, subfolder))

with open(os.path.join(folder, "prompts", f"layer{layer}_head{head}_prompts.txt"), "w") as f:
    f.write(f"--- Response 1 ---\n{assistant_response_1}\n\n--- Response 2 ---\n{assistant_response_2}")

python_snippet = assistant_response_2.split("```python")[1].split("```")[0].strip()
with open(os.path.join(folder, "llm_code", f"layer{layer}_head{head}_code.py"), "w") as f:
    f.write(python_snippet)

In [ ]:
#helpers

import json
import re
import textwrap

import re
import json
import textwrap

def parse_model_output(output_text: str):
    """
    Robustly parse model output with 'hypothesis' and 'program' keys.
    Handles triple quotes, code fences, and inconsistent indentation.
    Ignores extra text after the JSON object.
    """
    # 1️⃣ Remove outer markdown fences
    cleaned = re.sub(r"^```(?:json)?|```$", "", output_text.strip(), flags=re.MULTILINE).strip()

    # 2️⃣ Handle malformed JSON with triple quotes
    if '"""' in cleaned:
        hyp_match = re.search(r'"hypothesis"\s*:\s*"([^"]+)"', cleaned)
        prog_match = re.search(r'"program"\s*:\s*("""|```python|```)(.*?)(\1|```)', cleaned, re.DOTALL)
        if not hyp_match or not prog_match:
            raise ValueError("Could not locate hypothesis or program block.")
        hypothesis = hyp_match.group(1).strip()
        program_raw = prog_match.group(2)
        program = textwrap.dedent(program_raw).strip()
        program = program.replace('\r\n', '\n')
        return {"hypothesis": hypothesis, "program": program}

    # 3️⃣ Parse valid JSON normally, but extract only the first JSON object
    # This ignores any extra text after the first closing brace
    json_match = re.search(r"\{.*?\}\s*(?=\n|$)", cleaned, flags=re.DOTALL)
    if not json_match:
        raise ValueError("No JSON object found in model output.")

    json_str = json_match.group(0)

    try:
        data = json.loads(json_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON: {e}\nExtracted text:\n{json_str}")

    if not all(k in data for k in ("hypothesis", "program")):
        raise KeyError(f"Missing keys. Found: {list(data.keys())}")

    # Clean code block & indentation
    program_raw = re.sub(r"^```(?:python)?|```$", "", data["program"].strip(), flags=re.MULTILINE)
    program = textwrap.dedent(program_raw).strip().replace('\r\n', '\n')

    return {
        "hypothesis": data["hypothesis"].strip(),
        "program": program,
    }

import traceback
import importlib.util
import types
def validate_program(program_path, model, tokenizer, layer, head, sentences):       
    try:
        spec = importlib.util.spec_from_file_location("loaded_program", program_path)
        module = importlib.util.module_from_spec(spec)
        module.__dict__['np'] = np
        spec.loader.exec_module(module)
    except Exception as e:
        print(f"Program loading failed: {str(e)}")
        return str(e)

    for attr_name in dir(module):
        attr = getattr(module, attr_name)
        if isinstance(attr, types.FunctionType):
            program_to_test = attr
            break

    try:
        # print("Scoring program...")
        score = score_prediction(model, tokenizer, (layer, head), program_to_test, sentence, distance="jsd", output=False)
        return score
    except Exception as e:
        error = traceback.format_exc()
        full_error = f"Program validation failed: {str(e)} +\n{error}"
        return full_error

In [ ]:
model = AutoModel.from_pretrained("gpt2", output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

#print name, num heads, num layers, 
print(model.config.architectures[0])
print(model.config.num_attention_heads)
print(model.config.num_hidden_layers)

In [ ]:
model = AutoModel.from_pretrained("bert-base-uncased", output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

torch_model = AutoModel.from_pretrained("bert-base-uncased", output_attentions=True)
torch_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

num_layers = model.config.num_hidden_layers
num_heads = model.config.num_attention_heads

for layer in range(model.config.num_hidden_layers):
    for head in range(model.config.num_attention_heads):
        if (layer, head) in [(0, 1), (0, 2), (0, 3), (0, 4), (0,5)]: continue

        if sq_score[layer, head] < 0.7: continue
        
        # if (layer, head) not in fails: continue

        success = False

        print(f"Layer {layer}, Head {head}")
        # max_attempts = 3  # Initial try + two more attempt 
        head_attempts = 0

        # while head_attempts < max_attempts:
        if success: break

        try:
            # from automation_helper import generate_prompt, validate_program
            # print("got here")
            fullprompt = generate_prompt(sentences, model, tokenizer, (layer, head), top_k_ratio=0.025)

            conversation_history = [
                {"role": "system",
                "content": "You are a coding assistant with linguistic expertise."}
            ]
            conversation_history.append({"role": "user", "content": fullprompt})
            response_1 = client.chat.completions.create(
                model="gpt-4o", messages=conversation_history
            )
            assistant_response_1 = response_1.choices[0].message.content
            conversation_history.append({"role": "assistant", "content": assistant_response_1})
            print(f"--- Response 1 ---\n{assistant_response_1}")
            try:
                parsed = parse_model_output(assistant_response_1)
            except:
                continue
            feedback = "invalid_output"

            folder = "automation_results_bert_2"
            if not os.path.exists(folder): os.makedirs(folder)
            subfolders = ["prompts", "llm_code", "scores"]
            for subfolder in subfolders:
                if not os.path.exists(os.path.join(folder, subfolder)):
                    os.makedirs(os.path.join(folder, subfolder))

            hypothesis_path = f"{folder}/prompts/layer{layer}_head{head}_prompts.txt"
            with open(hypothesis_path, "w") as f: f.write(parsed["hypothesis"])

            python_path = f"{folder}/llm_code/layer{layer}_head{head}_code.py"
            with open(python_path, "w") as f: f.write(parsed["program"].rstrip("}"))

            import traceback
            import importlib.util
            import types
            def validate_program(program_path, model, tokenizer, layer, head, sentences):       
                try:
                    spec = importlib.util.spec_from_file_location("loaded_program", program_path)
                    module = importlib.util.module_from_spec(spec)
                    module.__dict__['np'] = np
                    spec.loader.exec_module(module)
                except Exception as e:
                    print(f"Program loading failed: {str(e)}")
                    return str(e)

                for attr_name in dir(module):
                    attr = getattr(module, attr_name)
                    if isinstance(attr, types.FunctionType):
                        program_to_test = attr
                        break

                try:
                    # print("Scoring program...")
                    head_scores = []
                    for sentence in sentences[:10]:
                            score = score_prediction(model, tokenizer, (layer, head), program_to_test, sentence, distance="jsd", output=False)
                            head_scores.append(score)
                    score = np.mean(head_scores)
                    return score
                except Exception as e:
                    error = traceback.format_exc()
                    full_error = f"Program validation failed: {str(e)} +\n{error}"
                    return full_error

            max_refinements = 2 # no refinements
            while type(feedback) is str or feedback > 0.7:
                if max_refinements >= 3: break
                
                feedback = validate_program(python_path, model, tokenizer, layer, head, sentences)
                print(feedback)
                if isinstance(feedback, np.float64) and feedback <= 0.7:
                    with open(python_path, "w") as f: f.write(parsed["program"].rstrip("}"))
                    success = True
                    break

                if type(feedback) is str:
                    conversation_history.append({"role": "user", "content": f"The following error was encountered when running your code: {feedback}. Please fix your code and return json with two keys: 'hypothesis' and 'program', where hypothesis is the explanation of what you fixed and program is the updated program. Here is your previous code: {parsed['program']}"})
                else:
                    conversation_history.append({"role": "user", "content": f"The score of your program was {feedback:.2f}, which is not good enough. Please refine your code and return json with two keys: 'hypothesis' and 'program', where hypothesis is the explanation of what you fixed and program is the updated program. Here is your previous code: {parsed['program']}"})
                response_2 = client.chat.completions.create(
                    model="gpt-4o",
                    messages=conversation_history
                )

                try:
                    parsed = parse_model_output(response_2.choices[0].message.content)
                except Exception as e:
                    print(f"Error parsing response: {e}")
                    continue

                with open(os.path.join(folder, "prompts", f"layer{layer}_head{head}_prompts_try{max_refinements}.txt"), "w") as f:
                    f.write(parsed["hypothesis"])
                
                max_refinements += 1

            score = feedback
            score_path = f"{folder}/scores/layer{layer}_head{head}_score.txt"
            with open(score_path, "w") as f: f.write(str(score))

        except Exception as e:
            print(f"Error processing Layer {layer}, Head {head}: {e}")
            head_attempts += 1
            continue

In [ ]:
# check that gpt_call works as expected

prompt = generate_prompt(sentences[:25], model, tokenizer, (3, 9), top_k_ratio=0.025)
conversation_history = [
    {"role": "system",
    "content": "You are a coding assistant with linguistic expertise."}
]
conversation_history.append({"role": "user", "content": prompt})
response = client.chat.completions.create(
    model="gpt-4o", messages=conversation_history
)
assistant_response = response.choices[0].message.content
print(assistant_response)
try:
    output = parse_model_output(assistant_response)
except:
    output = "parsing_error"
    
print(output)

In [ ]:
def gpt_call(prompt):
    conversation_history = [
        {"role": "system",
        "content": "You are a coding assistant with linguistic expertise."}
    ]
    conversation_history.append({"role": "user", "content": prompt})
    response = client.chat.completions.create(
        model="gpt-4o", messages=conversation_history
    )
    assistant_response = response.choices[0].message.content
    # print(assistant_response)
    return assistant_response
    
def make_program_executable(program_str: str) -> Callable:
    import importlib.util
    import types
    import tempfile
    import os

    with tempfile.NamedTemporaryFile(delete=False, suffix=".py") as temp_file:
        temp_file.write(program_str.encode())
        temp_file_path = temp_file.name

    try:
        spec = importlib.util.spec_from_file_location("loaded_program", temp_file_path)
        module = importlib.util.module_from_spec(spec)
        module.__dict__['np'] = np
        module.__dict__['PreTrainedTokenizerBase'] = PreTrainedTokenizerBase
        module.__dict__['Optional'] = Optional
        module.__dict__['Tuple'] = Tuple
        module.__dict__['Callable'] = Callable
        module.__dict__['spacy'] = spacy
        spec.loader.exec_module(module)
    finally:
        os.remove(temp_file_path)

    for attr_name in dir(module):
        attr = getattr(module, attr_name)
        if isinstance(attr, types.FunctionType):
            return attr

    raise ValueError("No function found in the provided program.")

def diff(program, model, tokenizer, sentences):
    # take a short list (5 examples for each failed program) of the token pairs with the biggest difference between the actual activations and the hypothesis matrix.
    # return a list of tuples (token1, token2, actual_score, predicted_score)
    differences = []
    # activation is a matrix i, j of actual attention scores
    # program is a function that takes in a sentence and tokenizer and returns a matrix of predicted

    for sentence in sentences:
        tokens = tokenizer(sentence, return_tensors="pt")
        activations = model(**tokens, output_attentions=True).attentions[layer][0, head].detach().cpu().numpy()

        predicted = program(sentence, tokenizer)[1]

        seq_len = activations.shape[0]
        for i in range(seq_len):
            for j in range(seq_len):
                actual_score = activations[i, j]
                predicted_score = predicted[i, j]
                token_i = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0][i].item())
                token_j = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0][j].item())
                differences.append([token_i, token_j, actual_score, predicted_score])

    differences = sorted(differences, key=lambda x: abs(x[2] - x[3]), reverse=True)
    differences = differences[:10]
    actual_scores = [f"{diff[2]:.4f}" for diff in differences]
    predicted_scores = [f"{diff[3]:.4f}" for diff in differences]
    differences = [[diff[0], diff[1], actual, predicted] for diff, actual, predicted in zip(differences, actual_scores, predicted_scores)]
    return differences

In [ ]:
# --------------- GREEDY REFINEMENT PIPELINE ---------------

model_name = "bert-base-uncased"
model = AutoModel.from_pretrained(model_name, output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
from automation_helper import generate_prompt

save_path = "automation_results_bert_greedy"
subdirectories = ["llm_code", "scores"]
for subdir in subdirectories:
    dir_path = os.path.join(save_path, subdir)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

for layer in range(model.config.num_hidden_layers):
    for head in range(model.config.num_attention_heads):
        if (layer, head) in [(0,0), (0,1), (0,2), (0,3)]: continue
        try:

            score_1_path = os.path.join("automation_results_bert", "scores", f"layer{layer}_head{head}_score.txt")
            score_2_path = os.path.join("automation_results_bert_2", "scores", f"layer{layer}_head{head}_score.txt")
            score_1 = float(open(score_1_path).read()) if os.path.exists(score_1_path) else 1.0
            score_2 = float(open(score_2_path).read()) if os.path.exists(score_2_path) else 1.0
            if score_1 < 0.4 or score_2 < 0.4:
                print(f"Skipping Layer {layer}, Head {head} due to existing low score ({min(score_1, score_2):.2f})")
                continue

            print(f"\nAnalyzing Layer {layer}, Head {head} of [{model.config.architectures[0]} - {model_name}]")
            prompt = generate_prompt(sentences[:25], model, tokenizer, (layer, head), top_k_ratio=0.025)
            raw_response = gpt_call(prompt)
            try:
                assistant_response = parse_model_output(raw_response)
            except:
                max_retries = 0
                while max_retries < 3:
                    print("\tParsing error encountered. Retrying...")
                    assistant_response = gpt_call(prompt)
                    if assistant_response != "parsing_error": break
                    max_retries += 1

                if max_retries == 3: 
                    print(f"\tFailed to parse after {max_retries} attempts. Skipping Layer {layer}, Head {head}.")
                    continue
            program = assistant_response['program']
            program_path = os.path.join(save_path, "llm_code", f"layer{layer}_head{head}_code.py")
            with open(program_path, "w") as f: f.write(program.rstrip("}"))
            feedback = validate_program(program_path, model, tokenizer, layer, head, sentences)
            print(f"\tProgram parsed successfully. Feedback: {feedback}")

            retries = 0
            failed_programs = []
            while (isinstance(feedback, float) and feedback > 0.4) or (isinstance(feedback, str)) and retries < 10:
                if isinstance(feedback, float) and feedback <= 0.4: break

                retries += 1
                refinement = ""
                if isinstance(feedback, float):
                    try:
                        executable_program = make_program_executable(program)
                        failed_programs.append(executable_program.__name__)
                        tokens = tokenizer(sentence, return_tensors="pt")
                        activations = model(**tokens, output_attentions=True).attentions[layer][0, head].detach().cpu().numpy()
                        top_differences = diff(executable_program, model, tokenizer, sentences[:25])
                        refinement += f"\nPrevious failed programs: {failed_programs}. The top 10 differences in form (token1, token2, actual_score, predicted_score): {top_differences}"
                    except Exception as e:
                        continue
                
                raw_response = gpt_call(prompt + refinement)
                assistant_response = parse_model_output(raw_response)
                try:
                    program = assistant_response['program']
                except Exception as e:
                    print(f"\tParsing error on refinement {retries}: {e}")
                    feedback = "parsing_error"
                    continue
                program_path = os.path.join(save_path, "llm_code", f"layer{layer}_head{head}_code.py")
                with open(program_path, "w") as f: f.write(program.rstrip("}"))
                feedback = validate_program(program_path, model, tokenizer, layer, head, sentences)

            score = feedback
            llm_code_path = os.path.join(save_path, "llm_code", f"layer{layer}_head{head}_code.py")
            score_path = os.path.join(save_path, "scores", f"layer{layer}_head{head}_score.txt")
            with open(llm_code_path, "w") as f: f.write(program.rstrip("}"))
            with open(score_path, "w") as f: f.write(str(score))
            print(f"\tFinal score for Layer {layer}, Head {head}: {score}")

        except Exception as e:
            print(f"\tError processing Layer {layer}, Head {head}: {e}")
            continue


In [ ]:
differences = []
    # activation is a matrix i, j of actual attention scores
    # program is a function that takes in a sentence and tokenizer and returns a matrix of predicted

for sentence in sentences[:25]:
    tokens = tokenizer(sentence, return_tensors="pt")
    activations = model(**tokens, output_attentions=True).attentions[layer][0, head].detach().cpu().numpy()
    predicted = executable_program(sentence, tokenizer)[1]

    seq_len = activations.shape[0]
    for i in range(seq_len):
        for j in range(seq_len):
            actual_score = activations[i, j]
            predicted_score = predicted[i, j]
            token_i = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0][i].item())
            token_j = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0][j].item())
            differences.append([token_i, token_j, actual_score, predicted_score])

differences = sorted(differences, key=lambda x: abs(x[2] - x[3]), reverse=True)
differences = differences[:10]
actual_scores = [f"{diff[2]:.4f}" for diff in differences]
predicted_scores = [f"{diff[3]:.4f}" for diff in differences]
differences = [[diff[0], diff[1], actual, predicted] for diff, actual, predicted in zip(differences, actual_scores, predicted_scores)]

In [ ]:
print(seq_len)
for i in range(seq_len):
    print("-----")
    for j in range(seq_len):
        print(activations[i,j])
        # how to get the token corresponding to index i and j
        print(tokenizer.convert_ids_to_tokens(tokens['input_ids'][0][i].item()), tokenizer.convert_ids_to_tokens(tokens['input_ids'][0][j].item()))

In [ ]:
print(actual_score, predicted_score, diff)

In [ ]:
#print the code that makes up executable_program
import inspect
print(inspect.getsource(executable_program))

In [ ]:
# iterate through scores folder and count how many are floats and how many are strings
# edit the following code to count nan as a failure

folder = r"automation_results_gpt2\scores"
float_count = 0
string_count = 0
nan_count = 0
no_code_count = 0

fails = []

for i in range(12):
    for j in range(12):
        layer, head = i, j
        if not os.path.exists(os.path.join(folder, f"layer{layer}_head{head}_score.txt")):
            fails.append((layer, head))
            no_code_count += 1

for filename in os.listdir(folder):
    if filename.endswith("_score.txt"):
        with open(os.path.join(folder, filename), "r") as f:
            content = f.read().strip()
            try:
                value = float(content)
                if value != value:  # Check for NaN
                    nan_count += 1
                    layer = int(filename.split("_")[0][5:])
                    head = int(filename.split("_")[1][4:])
                    fails.append((layer, head))
                else:
                    float_count += 1
            except ValueError:
                layer = int(filename.split("_")[0][5:])
                head = int(filename.split("_")[1][4:])
                fails.append((layer, head))
                string_count += 1

print(f"Float scores: {float_count}")
print(f"String scores: {string_count}")
print(f"NaN scores: {nan_count}")
print(f"No code scores: {no_code_count}")
print(f"Total scores: {float_count + string_count + nan_count + no_code_count}\n\n")
print(fails)

In [ ]:
# retest the bert scores using existing functions with 100 sentences for each head

retest = [(11, 10), (4,11)]

scores = []
for layer in range(model.config.num_hidden_layers):
    for head in range(model.config.num_attention_heads):

        if (layer, head) not in retest: continue

        print(f"Layer {layer}, Head {head}")
        # find file in llm_code e.g. layer0_head0_code
        program_path = f"automation_results_bert/llm_code/layer{layer}_head{head}_code.py"

        try:
            spec = importlib.util.spec_from_file_location("loaded_program", program_path)
            module = importlib.util.module_from_spec(spec)
            module.__dict__['np'] = np
            spec.loader.exec_module(module)
        except Exception as e:
            print(f"Program loading failed: {str(e)}")
            continue

        for attr_name in dir(module):
            attr = getattr(module, attr_name)
            if isinstance(attr, types.FunctionType):
                program_to_test = attr
                break

        head_scores = []
        for sentence in sentences[:100]:
            try:
                score = score_prediction(model, tokenizer, (layer, head), program_to_test, sentence, distance="jsd", output=False)
                head_scores.append(score)
            except Exception as e:
                error = traceback.format_exc()
                full_error = f"Program validation failed: {str(e)} +\n{error}"
                continue
        
        scores.append(np.mean(head_scores))
        print(f"Average Score for Layer {layer}, Head {head}: {np.mean(head_scores)}")
        # overwrite score file
        score_path = f"automation_results_bert/scores/layer{layer}_head{head}_score.txt"
        with open(score_path, "w") as f: 
            f.write(str(np.mean(head_scores)))

In [ ]:
# folder = r"automation_results_bert\scores"
# scores = []

# for filename in os.listdir(folder):
#     if filename.endswith("_score.txt"):
#         with open(os.path.join(folder, filename), "r") as f:
#             content = f.read().strip()
#             try:
#                 value = float(content)
#                 scores.append(round(value, 3))
#             except ValueError:
#                 continue

# scores = np.array(scores, dtype=float)
# scores[np.isnan(scores)] = np.mean(scores[~np.isnan(scores)])

# def find_outliers(data):
#     data = np.array(data)
#     Q1 = np.percentile(data, 25)
#     Q3 = np.percentile(data, 75)

#     IQR = Q3 - Q1
#     lower_bound = Q1 - (1.5 * IQR)
#     upper_bound = Q3 + (1.5 * IQR)

#     outliers = data[(data < lower_bound) | (data > upper_bound)]
#     return outliers, lower_bound, upper_bound, Q1, Q3, IQR

# outliers, lower_bound, upper_bound, Q1, Q3, IQR = find_outliers(scores)

def plot_scores_boxplot(scores):
    plt.figure(figsize=(3.2,8))
    plt.boxplot(
        scores,
        positions=[0.75], 
        vert=True,
        patch_artist=True,
        medianprops={'color': 'black', 'linewidth': 3},
        boxprops={'facecolor': 'gray', 'edgecolor': 'black'},
        flierprops={'marker': 'D', 'markerfacecolor': 'black', 'markersize': 3, 'linestyle': 'none'}
    )

    # plt.title(f'Auto K=N | GPT2\nMean={np.mean(scores):.2f}\n\nLasso (alpha=0.001)', fontsize=14, weight='bold')
    plt.title(f'Auto K=N | GPT \n\nRandom Token Baseline\navg_score={np.mean(scores):.2f}', fontsize=14, weight='bold')
    plt.ylabel('Automation Scores', fontsize=12)
    plt.xticks([])
    plt.ylim(0, 1.05)

    #insert the text 'WIP' in center of plot
    # plt.text(0.75, 0.5, 'WIP', fontsize=12, ha='center', va='center')

    plt.grid(axis='y', linestyle='--', alpha=0.6)
    x = np.ones_like(scores)
    plt.scatter(
        x,
        scores,
        color='gray',
        edgecolor='black',
        s=30,
        alpha=0.9,
    )
    plt.show()

# scores = np.load("data/interpolation_experiment_gpt2_lasso_a=0.001_k=N.npy")
scores_flat = np.array(scores).flatten()
plot_scores_boxplot(scores_flat)

In [ ]:
# loop through "C:\Users\amkah\OneDrive\Documents\GitHub\LLM-Interpretability\automation_refinement_gpt2\master_list"
# if python file is empty or only has new lines, delete that file

folder = r"C:\Users\amkah\OneDrive\Documents\GitHub\LLM-Interpretability\automation_refinement_gpt2\master_list"
for filename in os.listdir(folder):
    if filename.endswith(".py"):
        file_path = os.path.join(folder, filename)
        if os.path.getsize(file_path) == 0 or all(line.strip() == "" for line in open(file_path)):
            os.remove(file_path)
            print(f"Deleted empty file: {filename}")

In [ ]:
from programs import *
patterns = [next_attention, previous_attention, same_attention, last_token_attention, repeated_attention, uniform_attention, cls_attention, eos_attention, special_token_attention, pos_alignment, dependencies]

number_of_lines = []
# first, count number of lines in existing patterns
import inspect as i
for pattern in patterns:
    definition = i.getsource(pattern)
    number_of_lines.append(len(definition.strip().split('\n')))

master_list_dir = "automation_refinement/master_list"
for filename in os.listdir(master_list_dir):
    if filename.endswith(".py"):
        module_name = filename[:-3]
        if module_name not in [p.__name__ for p in patterns]:
            try:
                program_path = os.path.join(master_list_dir, filename)
                spec = importlib.util.spec_from_file_location("loaded_program", program_path)
                module = importlib.util.module_from_spec(spec)
                module.__dict__['np'] = np
                # module.__dict__['torch'] = torch
                module.__dict__['spacy'] = spacy
                module.__dict__['nlp'] = nlp
                module.__dict__['PreTrainedTokenizerBase'] = PreTrainedTokenizerBase
                module.__dict__['Optional'] = Optional
                module.__dict__['Tuple'] = Tuple
                module.__dict__['Callable'] = Callable
                module.__dict__['re'] = re
                spec.loader.exec_module(module)
            except Exception as e:
                print(f"Program loading failed: {str(e)}")

            for attr_name in dir(module):
                attr = getattr(module, attr_name)
                if isinstance(attr, types.FunctionType):
                    program_to_test = attr
                    break
            
            # count number of lines in program_to_test and append
            with open(program_path, 'r') as f:
                lines = f.readlines()
                number_of_lines.append(len(lines))

            patterns.append(program_to_test)

print(f"Total patterns to analyze: {len(patterns)}")
for pattern in patterns: print(f"\tPattern: {pattern.__name__}")

In [ ]:
interpolation_matrix

In [ ]:
# --------------- LINEAR INTERPOLATION REFINEMENT PIPELINE ---------------
import inspect
import warnings
warnings.filterwarnings("ignore")

# model = AutoModel.from_pretrained("bert-base-uncased", output_attentions=True)
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("openai-community/gpt2", output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
torch_model = model
torch_tokenizer = tokenizer

num_layers = model.config.num_hidden_layers
num_heads = model.config.num_attention_heads

interpolation_matrix = np.zeros((num_layers, num_heads))
# patterns = [previous_attention, same_attention, punctuation_attention, last_token_attention, repeated_attention, uniform_attention, cls_attention, special_token_attention, pos_alignment, relative_position_attention]

folder = r"automation_refinement_gpt2_try2"
subfolders = ["master_list", "candidate"]
if not os.path.exists(folder): os.makedirs(folder)
for subfolder in subfolders:
    if not os.path.exists(os.path.join(folder, subfolder)):
        os.makedirs(os.path.join(folder, subfolder))

for pattern in patterns:
    pattern_name = pattern.__name__
    with open(os.path.join(folder, "master_list", f"{pattern_name}.py"), "w") as f:
        f.write(inspect.getsource(pattern))

new_patterns = 0
total_loops = 0
while new_patterns < 30 and total_loops < 100:
    try:
        total_loops += 1
        print(f"Searching for new patterns, iteration {new_patterns} | total loops {total_loops}...")

        # get interpolation scores for all heads
        print(f"\tCalculating interpolation scores for all heads...")
        for layer in range(num_layers):
            if layer % 3 == 0: print(f"\t\tLayer {layer}/{num_layers} complete")
            for head in range(num_heads):
                sentence = str(np.random.choice(sentences))

                X = []
                for pattern in patterns:
                    try:
                        instance = pattern(sentence, tokenizer)[1].flatten()
                        # check if nan in instance:
                        if np.isnan(instance).any():
                            del patterns[patterns.index(pattern)]
                            print(f"\t\tRemoved pattern {pattern.__name__} due to NaN values.")
                            continue
                        # print(f"Tested pattern: {pattern.__name__} successfully")

                    except Exception as e:
                        # print(f"Error testing pattern {pattern.__name__}: {e}")
                        print(f"\t\tRemoved pattern {pattern.__name__} due to error during testing.")
                        del patterns[patterns.index(pattern)]
                        continue

                    X.append(instance)
                X_n = np.nan_to_num(np.array(X).T)
                y = np.nan_to_num(model(**tokenizer(sentence, return_tensors="pt"), output_attentions=True).attentions[layer][0, head].detach().numpy().flatten())

                reg = LinearRegression().fit(X_n, y.flatten())
                out = reg.intercept_ + sum(coef * mat for coef, mat in zip(reg.coef_, X))
                len_seq = len(tokenizer([sentence], return_tensors="pt").input_ids[0])
                out = out.reshape((len_seq, len_seq))
                pred_att = out / out.sum(axis=1, keepdims=True)

                tokens = tokenizer(sentence, return_tensors="pt")
                att = model(**tokens, output_attentions=True).attentions[layer][0, head].detach().numpy()

                jensonshannon_distances = []
                for row_att, row_out in zip(att, pred_att):
                    jensonshannon_distances.append(np.sqrt(js_divergence(row_att, row_out)))
                interpolation_score = np.mean(jensonshannon_distances)

                interpolation_matrix[layer, head] = interpolation_score
                # print(f"Layer {layer}, Head {head} - Interpolation Score: {interpolation_score:.4f}")

        # select worst 20% of heads based on interpolation score
        sorted_indices = np.dstack(np.unravel_index(np.argsort(interpolation_matrix.ravel()), interpolation_matrix.shape))[0]
        num_to_select = int(0.1 * num_layers * num_heads)
        worst_indices = sorted_indices[-num_to_select:]

        # generate candidate programs for each of the worst heads
        candidate_scores = []
        candidate_paths = []
        for layer, head in worst_indices[::-1]: # start with worst head
            layer, head = int(layer), int(head)
            print(f"\t TRYING | Layer {layer}, Head {head} - Previous Interpolation Score: {interpolation_matrix[layer, head]:.4f}")

            try:
                fullprompt = generate_prompt(generic_sentences[:25], model, tokenizer, (layer, head), top_k_ratio=0.025)
                refinement_part = "It's already been found that this head doesn't do well on any of the following patterns: "
                refinement_part += ", ".join([pattern.__name__ for pattern in patterns])
                refinement_part += ". So don't suggest any of those patterns again."
                fullprompt += " ".join(refinement_part)

                conversation_history = [
                    {"role": "system",
                    "content": "You are a coding assistant with linguistic expertise."}
                ]
                conversation_history.append({"role": "user", "content": fullprompt})
                response_1 = client.chat.completions.create(
                    model="gpt-4o", messages=conversation_history
                )
                assistant_response_1 = response_1.choices[0].message.content
                conversation_history.append({"role": "assistant", "content": assistant_response_1})
                # print(f"--- Response 1 ---\n{assistant_response_1}")
                try:
                    parsed = parse_model_output(assistant_response_1)
                except:
                    print(f"\t\tError parsing response, skipping head ({layer}, {head})")
                    continue
                feedback = "invalid_output"

                # print(f"\t\t{parsed}")
                print(f"\t\tHypothesis: {parsed['hypothesis']}. Program successfully parsed.")

                candidate_path = f"{folder}/candidate/layer{layer}_head{head}_code.py"
                with open(candidate_path, "w") as f: f.write(parsed["program"].rstrip("}"))

                feedback = validate_program(candidate_path, model, tokenizer, layer, head, sentences)
                if isinstance(feedback, np.float64) and not np.isnan(feedback):
                    candidate_scores.append(feedback)
                    candidate_paths.append(candidate_path)
                    print(f"\t\tProgram valid with score {feedback:.4f}.")
                    continue
                    
                else:
                    candidate_scores.append(100)
                    candidate_paths.append("")
                    # print(f"\t\tInitial program invalid w/ error: ({feedback}), skipping.")
                    continue
            
            except Exception as e:
                total_loops += 1
                candidate_scores.append(100)
                candidate_paths.append("")
                # print(f"\t\tError processing Layer {layer}, Head {head}: {e}")
                continue

        # select and save best candidate program to master list
        best_candidate = candidate_paths[np.argmin(candidate_scores)]
        best_score = np.min(candidate_scores)
        new_program = make_program_executable(open(best_candidate).read())
        
        print(f"Best candidate program {new_program.__name__} added | solo score {best_score:.4f}.")

        if best_score < 0.45:
            spec = importlib.util.spec_from_file_location("loaded_program", best_candidate)
            module = importlib.util.module_from_spec(spec)
            module.__dict__['np'] = np
            spec.loader.exec_module(module)

            for attr_name in dir(module):
                attr = getattr(module, attr_name)
                if isinstance(attr, types.FunctionType):
                    new_program = attr

            new_pattern_name = new_program.__name__
            with open(os.path.join(folder, "master_list", f"{new_pattern_name}.py"), "w") as f:
                f.write(inspect.getsource(new_program))

            patterns.append(new_program)
            new_patterns += 1

    except Exception as e:
        total_loops += 1
        print(f"Error in main loop: {e}")
        continue

In [ ]:
best_candidate = r"automation_refinement_gpt2_try2/candidate/layer2_head2_code.py"
make_program_executable(open(best_candidate).read())

In [ ]:
for pattern in patterns:
    try:
        instance = pattern(sentence, tokenizer)[1].flatten()
        # check if nan in instance:
        if np.isnan(instance).any():
            del patterns[patterns.index(pattern)]
            print(f"\t\tRemoved pattern {pattern.__name__} due to NaN values.")
            continue
        print(f"Tested pattern: {pattern.__name__} successfully")

    except Exception as e:
        # print(f"Error testing pattern {pattern.__name__}: {e}")
        del patterns[patterns.index(pattern)]
        continue

In [ ]:
# get all programs from automation_refinement/master_list and load these python functions as patterns = [executable functions]

import os
import importlib.util
import types
import numpy as np
from transformers import AutoTokenizer, AutoModel
import regex as re
import pandas as pd
import spacy
from transformers import PreTrainedTokenizerBase, PreTrainedModel
from typing import Optional, Tuple, Callable
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

folder = "automation_refinement_gpt2/master_list"
patterns = []

for filename in os.listdir(folder):
    if filename.endswith(".py"):
        code_path = os.path.join(folder, filename)
        spec = importlib.util.spec_from_file_location(f"module_{filename[:-3]}", code_path)
        module = importlib.util.module_from_spec(spec)
        module.__dict__['np'] = np
        # get pretrainedtokenizerbase, from typing import Optional, Tuple, Callable
        module.__dict__['PreTrainedTokenizerBase'] = PreTrainedTokenizerBase
        module.__dict__['Optional'] = Optional
        module.__dict__['Tuple'] = Tuple
        module.__dict__['Callable'] = Callable
        module.__dict__['spacy'] = spacy
        
        try:
            spec.loader.exec_module(module)
            for attr_name in dir(module):
                attr = getattr(module, attr_name)
                if isinstance(attr, types.FunctionType):
                    patterns.append(attr)
                    break
        except Exception as e:
            print(f"Error loading program from {filename}: {e}")
            continue

print(f"Loaded {len(patterns)} patterns.")
for i, prog in enumerate(patterns):
    print(f"{i}: {prog.__name__}")

In [ ]:
# go through patterns and check if they need nlp or defaultdict, if they don't need nlp library from spacy, add to patterns_strong
patterns_strong = []
for pattern in patterns:
    if not any(kw in pattern.__code__.co_names for kw in ['nlp', 'spacy', 'defaultdict']):
        patterns_strong.append(pattern)
print(f"Filtered to {len(patterns_strong)} strong patterns.")

In [ ]:
test_sentences = sentences[135:200]
# gpt2
model = AutoModel.from_pretrained("gpt2", output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

scores = np.zeros((model.config.num_hidden_layers, model.config.num_attention_heads))
top_k = 3

for layer in range(num_layers):
    for head in range(num_heads):
        print(f"Analyzing Layer {layer}, Head {head}...")
        sentence_scores = []
        for sentence in test_sentences:
            inputs = tokenizer(sentence, return_tensors="pt")
            outputs = model(**inputs, output_attentions=True)
            attention = outputs.attentions[layer][0, head].detach().numpy()
            y = attention.flatten()

            X = []
            for pattern in patterns_strong:
                X.append(pattern(sentence, tokenizer)[1].flatten())
            X_n = np.array(X).T
            y = y.flatten()

            # avoid ValueError: Input X contains NaN.
            X_n = np.nan_to_num(X_n)
            y = np.nan_to_num(y)

            reg = LinearRegression().fit(X_n, y)
            side_length = int(np.sqrt(len(y)))
            y = y.reshape((side_length, side_length))

            top_indices = np.argsort(np.abs(reg.coef_))[-top_k:]
            pred_att = reg.intercept_ + sum(reg.coef_[i] * X[i] for i in top_indices)
            pred_att = pred_att.reshape((side_length, side_length))

            if top_k == 1:
                #pred_att is just the single pattern with highest coef, it isn't equal to a sum at all
                fn_highest_coeff = patterns_strong[np.argmax(np.abs(reg.coef_))]
                pred_att = fn_highest_coeff(sentence, tokenizer)[1]
                print(f"\tUsing pattern: {fn_highest_coeff.__name__}")

            jensonshannon_distances = []
            for row_att, row_out in zip(y, pred_att):
                jensonshannon_distances.append(np.sqrt(js_divergence(row_att, row_out)))
            score = np.mean(jensonshannon_distances)
            sentence_scores.append(score)
        
        scores[layer, head] = np.mean(sentence_scores)
        print(f"\tScore for Layer {layer}, Head {head}: {scores[layer, head]}")

np.save("gpt2_k1_automation_scores.npy", scores)

In [ ]:
# CONDUCT MODEL LEVEL ANALYSIS / GET SUMMARY SCORE FOR WHOLE MODEL

def classify_model(method, sentences, torch_model, torch_tokenizer):
    if method == "linear_fit":
        patterns = [next_attention, previous_attention, same_attention, punctuation_attention, repeated_attention, pos_alignment]
    elif method == "best_fit":
        saved_file = pd.read_csv('data/best_fit_t5.csv')
    elif method == "automation":
        from automation_helper import generate_prompt, parse_llm_idea, validate_program

    num_layers = torch_model.config.num_hidden_layers
    num_heads = torch_model.config.num_attention_heads

    all_scores = []
    final_scores = []
    
    if method != "automation":
        for idx, sentence in enumerate(sentences):
            scores = np.zeros((num_layers, num_heads))
            for i in range(num_layers):
                for j in range(num_heads):
                    layer, head = i, j
                    inputs = torch_tokenizer(sentence, return_tensors="pt")
                    len_seq = len(torch_tokenizer([sentence], return_tensors="pt").input_ids[0])

                    X = []
                    # y =  torch_model(**inputs, output_attentions=True).attentions[layer][0, head].detach().numpy()
                    decoder_input_ids = tokens["input_ids"]
                    outputs = torch_model(input_ids=inputs["input_ids"], decoder_input_ids=decoder_input_ids, output_attentions=True)
                    y = outputs.encoder_attentions[layer][0, head].detach().numpy()

                    if method == "random_baseline":
                        pred_att = np.zeros((len_seq, len_seq))
                        pred_att[:, -1] = 1.0

                    elif method == "best_fit":
                        matching_rows = saved_file[(saved_file['i'] == i) & (saved_file['j'] == j)]
                        if not matching_rows.empty:
                            best_pattern = matching_rows.loc[matching_rows['Score'].idxmax(), 'Pattern']
                            func = globals()[best_pattern]
                            _, pred_att = func(sentence, tokenizer)
                        else:
                            out = np.random.rand(len_seq, len_seq)
                            pred_att =  out / out.sum(axis=1, keepdims=True)

                    elif method == "linear_fit":
                        for pattern in patterns:
                            X.append(pattern(sentence, torch_tokenizer)[1].flatten())
                        X_n = np.array(X).T
                        y = y.flatten()

                        reg = LinearRegression().fit(X_n, y)
                        side_length = int(np.sqrt(len(y)))
                        y = y.reshape((side_length, side_length))

                        pred_att = reg.intercept_ + sum(coef * mat for coef, mat in zip(reg.coef_, X))
                        pred_att = pred_att.reshape((side_length, side_length))

                    jensonshannon_distances = []
                    for row_att, row_out in zip(y, pred_att):
                        jensonshannon_distances.append(np.sqrt(js_divergence(row_att, row_out)))
                    score = np.mean(jensonshannon_distances)
                    scores[layer, head] = score
                    
            all_scores.append(scores)
            final_scores.append(np.sum(scores))
            print(f"Processed sentence #{idx}/{len(sentences)}: Score: {np.sum(scores):.2f}\n\t->'{sentence}'")
        
    elif method == "automation":
        
        prompt = generate_prompt(sentences, model, tokenizer, (layer, head), top_k_ratio=0.025)
        hypothesis, program = parse_llm_idea(prompt, config=config, verbalize=False)
        python_path = f"{program_path}/{head}_output.py"
        feedback = validate_program(python_path, model, tokenizer, layer, head, sentences)
        scores[layer, head] = feedback
            

    print(f"Final Score: {sum(final_scores) / len(final_scores)}")
    return all_scores, final_scores

methods = ["random_baseline", "best_fit", "linear_fit"]
final_scores = {"random_baseline": 0, "best_fit": 0, "linear_fit": 0}

for method in methods:
    print(f"\nAnalyzing method: {method}")
    _, scores = classify_model(method, sentences[:10], model, tokenizer)
    final_scores[method] = np.mean(scores)

print("\n",final_scores)

In [ ]:
# PLOT DIFFERENT SUMMARY SCORES FOR THE MODEL

max_score = model.config.num_hidden_layers * model.config.num_attention_heads
raw_scores = [111, 92, 62, 65, 56]
labels = ['Random \nToken Baseline', 'Automatic\nPrograms', 'K=1Refined\nPrograms', 'Best Fit\nPrograms', 'Linear Weight\nPrograms']
colors = ['darkred', 'darkblue', '#6aa84f', '#800080']

# Normalize scores: lower scores become higher bars
scores = [(score / max_score) for score in raw_scores]
fig, ax = plt.subplots(figsize=(8, 5))
bars = plt.bar(labels, scores, color=colors, width=0.6)

# Add text labels on top of bars
for bar, raw, norm in zip(bars, raw_scores, scores):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height + 0.02,
             f'{norm:.2f}\n[ {int(raw)} / {max_score} ]', ha='center', va='bottom', fontsize=14)
ax.set_facecolor('#F5F5F5')

plt.ylim(0, 1.0)
# plt.title('Normalized Error (1 - Score / Max Score)')
plt.xticks(fontsize=14)
plt.text(plt.xlim()[0]-0.7, plt.ylim()[1]+0.05, '[bad hypotheses]', ha='left', va='bottom', fontsize=12, color='gray')
plt.text(plt.xlim()[0]-0.7, plt.ylim()[0]-0.13, '[well-fitting\nhypotheses]', ha='left', va='bottom', fontsize=12, color='gray')
plt.ylabel('Normalized Model Scores', fontsize=16, labelpad=20)
plt.grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

<div style="text-align: center; font-size: 2em;">
  <hr>
    Program Automation Efforts
  <hr>
</div>

In [ ]:
# AUTOMATICALLY GENERATE PATTERNS FOR A HEAD & GENERATE LLM PROMPT

example_program_one = """
def dependencies(sentence: str, tokenizer: PreTrainedTokenizerBase) -> Tuple[str, np.ndarray]: /
    toks = tokenizer([sentence], return_tensors="pt") /
    len_seq = len(toks.input_ids[0]) /
    out = np.zeros((len_seq, len_seq)) /
    words = sentence.split() /
    doc = nlp(" ".join(words)) /
    for stok in doc: /
        parent_index = stok.i /
        for child_stok in stok.children: /
            child_index = child_stok.i /
            out[parent_index+1, child_index+1] = 1 /
            out[child_index+1, parent_index+1] = 1 /
    out[0, 0] = 1 /
    out[-1, 0] = 1 /
    out += 1e-4 /
    out = out / out.sum(axis=1, keepdims=True) /
    return "Dependency Parsing Pattern", out /
"""
example_program_two = """
def same_attention(sentence: str, tokenizer: PreTrainedTokenizerBase) -> Tuple[str, np.ndarray]: /
    toks = tokenizer([sentence], return_tensors="pt")
    len_seq = len(toks.input_ids[0])
    out = np.zeros((len_seq, len_seq))
    for i in range(1, len_seq-1):
        out[i, i] = 1
    out[0,0] = 1
    out[-1,0] = 1
    return "Same Token Pattern", out
"""
example_program_three = """
def pos_alignment(sentence: str, tokenizer: PreTrainedTokenizerBase) -> Tuple[str, np.ndarray]:
    toks = tokenizer([sentence], return_tensors="pt") /
    len_seq = len(toks.input_ids[0]) /
    out = np.zeros((len_seq, len_seq)) /
    # assign toks, input_ids, word_ids, len_seq, out, doc /
    # use spacey to get pos_tags for tokens in docs [token.pos_ for token in doc] /
    # for token in pos_tags: /
    # loop through pos_tags and increment out[i,j] when pos_tags match /
    # assign cls (out[0, 0] = 1) and eos (out[-1, 0] = 1) to have self_attention /
    # Normalize out matrix by row (results in uniform attention) and return out /
    # return 'Part of Speech Implementation 1', out /
"""

def generate_prompt(sentences, model, tokenizer, head_loc, top_k_ratio=0.1):
    layer, head = head_loc
    data = {
        "layer": layer,
        "head": head,
        "model": model.config.architectures[0],
        "examples": []
    }

    def handle_score(score):
        # convert to percentage with 0 decimal places
        return "{:.0f}".format(score * 100)
        
    def scrape_head(att, tokens, top_k_ratio, ignore_special=True):
        seq_len = att.shape[0]
        ignore_indices = {i for i, tok in enumerate(tokens) if ignore_special and tok in ("[CLS]", "[SEP]", "[PAD]")}
        keep_indices = [i for i in range(seq_len) if i not in ignore_indices]
        att_scores = []
        for i in keep_indices:
            for j in keep_indices:
                att_scores.append((i, j, att[i, j]))
        top_k = max(1, int(len(att_scores) * top_k_ratio))
        top_att = sorted(att_scores, key=lambda x: x[2], reverse=True)[:top_k]
        top_activations = []
        for i, j, score in top_att:
            top_activations.append(f"[{str(tokens[i])}|{str(tokens[j])}:{handle_score(score)}]")
        top_activations_str = " ".join(top_activations).replace("[", "").replace("]", "")
        return top_activations_str
    
    for idx, sentence in enumerate(sentences):
        inputs = tokenizer(sentence, return_tensors="pt")
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        with torch.no_grad():
            outputs = model(**inputs, output_attentions=True)
            att = outputs.attentions[layer][0, head]
        att = att.detach().cpu().numpy()
        top_activations = scrape_head(att, tokens, top_k_ratio=top_k_ratio)
        item = {f"sentence {idx}": " ".join(tokens), "sentence attention": top_activations}
        data["examples"].append(item)

    data = json.dumps(data, indent=2)
    prompt = f"""
    Using the following pieces of data based on {len(sentences)} sentences, generate three hypothesises about the linguistic role the following head is responsible for based on patterns
    in the {model.config.architectures[0]} activations.  Then, choose the most fitting hypothesis for the head function using examples from the data. Finally, using the linguistic hypothesis you determine, 
    write a python function which takes in a sentence and tokenizer as parameters and outputs the name of the pattern you hypothesize along with a predicted_matrix (size: token_len * token_len), which is the 
    rule encoded matrix mirroring attention patterns you'd predict for any given sentence for Layer {layer}, Head {head}. Feel free to encode complex functions but write the simplest algorithm that captures your 
    observed pattern. You must respond to this prompt in JSON in the form "{{"hypothesis": "...", "program": "..."}} with your chosen hypothesis. Think carefully before generating any code.
    The first portion of your response has key "hypothesis" with the title of the hypothesis and the second portion of your response with key "program" should have valid python code starting with ```python and including imports. These patterns can be simple or 
    complex.  For uniformity, the first three lines of your function should be 'toks = tokenizer([sentence], return_tensors="pt") len_seq = len(toks.input_ids[0]) out = np.zeros((len_seq, len_seq))'.
    Make sure the token sequences from your tokenizer and spaCy (if you must use spaCy) are aligned via a dictionary if necessary, because they split text differently. Make sure you generalize your hypothesis pattern to any sentence. Functions can almost 
    always be expressed in fewer than 50 lines of code. As examples, it has been discovered one head is responsible for the complex task of dependency parsing. It's simplistic predicted pseudocode looks like: 
    {example_program_one}. Example 2: '''{example_program_two}''' Example 3: '''{example_program_three}'''. DATA: {data}"""
    return ' '.join(prompt.strip().split())

model_name = "bert-base-uncased"
model = AutoModel.from_pretrained(model_name, output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

layer, head = 5, 7
prompt = generate_prompt(generic_sentences[:25], model, tokenizer, (layer, head), 0.025)
print(prompt)

In [ ]:
# Using Gemini, GPT-4o, Claude, Deepseek
# API needs long contexts and free access
# Source to get API keys is "usage" key

load_dotenv()
API_CONFIGS = {
    "gemini": {
        "model": "gemini",
        "url": "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent",
        "key": os.getenv("GEMINI"),
        "headers_fn": lambda key: {"Content-Type": "application/json", "X-goog-api-key": key},
        "payload_fn": lambda prompt: {
            "contents": [{"parts": [{"text": prompt}]}],
            "generationConfig": {"response_mime_type": "application/json"}
        },
        "usage": "https://aistudio.google.com/apikey"
    },
    "openai": {
        "model": "openai",
        "url": "https://api.openai.com/v1/responses",
        "key": os.getenv("OPENAI"),
        "headers_fn": lambda key: {"Authorization": f"Bearer {key}", "Content-Type": "application/json"},
        "payload_fn": lambda prompt: {"model": "gpt-4.1", "input": prompt},
        "usage": "https://platform.openai.com/account/api-keys"
    },
    "claude": {
        "model": "claude",
        "url": "https://api.anthropic.com/v1/messages",
        "key": os.getenv("CLAUDE"),
        "headers_fn": lambda key: {"x-api-key": key, "Content-Type": "application/json", "Anthropic-Version":"2023-06-01"},
        "payload_fn": lambda prompt: {"model":"claude-sonnet-4-20250514", "messages":[{"role":"user","content":prompt}]},
        "usage": "https://platform.claude.com/api_keys"
    },
    "deepseek": {
        "model": "deepseek",
        "url": "https://api.deepseek.com/chat/completions",
        "key": os.getenv("DEEPSEEK"),
        "headers_fn": lambda key: {"Authorization": f"Bearer {key}", "Content-Type": "application/json"},
        "payload_fn": lambda prompt: {"model": "deepseek-chat", "input": prompt, "max_tokens": 1000},
        "usage": "https://platform.deepseek.com/api_keys"
    }
}

In [ ]:
# SAVE AUTOMATED HYPOTHESIS + VALIDATE GENERATED PROGRAM SYNTHESIS CODE

def parse_llm_idea(prompt, config="YOUR_API_CONFIG", verbalize=True):
    def make_request():
        headers = config["headers_fn"](config["key"])
        payload = config["payload_fn"](prompt)
        response = requests.post(config["url"], headers=headers, data=json.dumps(payload))
        response.raise_for_status()

        if config["model"] == "gemini":
            data = response.json()
            output = data["candidates"][0]["content"]["parts"][0]["text"]
        if config["model"] == "openai":
            pass
        if config["model"] == "claude":
            data = response.json()
            output = data["content"]["text"]
        if config["model"] == "deepseek":
            pass

        return output
    
    output = make_request()

    try:
        result = json.loads(output)

        if type(result) is list: result = result[0]
        hypothesis = result.get("hypothesis", "")
        program = result.get("program", "")

        if program.startswith("```python"): program = program[9:]
        if program.endswith("```"): program = program[:-3]
        program = program.strip()

        if verbalize: print("Hypothesis, Explanation & Program successfully parsed")

    except Exception as e:
        print(f"Parsing API failed: {str(e)}")
        return str(e)

    return hypothesis, program

config = API_CONFIGS["gemini"] 
parse_llm_idea(prompt, config=config, verbalize=True)

In [ ]:
# SIMPLE AUTOMATION OF PIPELINE FOR ANALYZING ALL HEADS & WRITING/SAVING PROGRAMS

def automation_pipeline(model, tokenizer, sentences, API_KEY, save_data=True, evaluate=False):
    heads = model.config.num_attention_heads
    layers = model.config.num_hidden_layers
    prompts, programs = [], []

    for layer in range(layers):
        # if layer == 0: continue
        if save_data:
            # save prompts:
            prompt_path = f"automation_2/prompts/{layer}/"
            os.makedirs(prompt_path, exist_ok=True)

            # save programs:
            program_path = f"automation_2/llm_code/code_layer_{layer}/"
            os.makedirs(program_path, exist_ok=True)

            # save scores:
            if evaluate:
                score_path = f"automation_2/scores/{layer}/"
                os.makedirs(score_path, exist_ok=True)

        for head in range(heads):
            # if head < 9: continue
            if (layer, head) not in failed_programs:
                continue
            prompt = generate_prompt(sentences, model, tokenizer, (layer, head), top_k_ratio=0.1)
            hypothesis, explanation, program = parse_llm_idea(prompt, API_KEY, output=False)
            print(f"Analyzed Layer {layer}, Head {head} | Hypothesis ~ {hypothesis} ")

            prompts.append(prompt)
            programs.append(program)

            if save_data:
                with open(f"{prompt_path}/{layer}_{head}_prompt.txt", "w") as f: f.write(prompt)
                with open(f"{program_path}/{head}_output.py", "w") as f: f.write(program)

        if evaluate: 
            pass

    return None

automation_pipeline(model, tokenizer, generic_sentences[:10], API_KEY=API_KEY, save_data=True, evaluate=False)

In [ ]:
# CALCULATE AND SAVE SCORES FOR AUTOMATICALLY GENERATED PROGRAMS

import importlib.util
import types

scores = []
failed_programs = []
for layer in range(12):
    # if layer != 11: continue
    code_path = f"automation_2/llm_code/code_layer_{layer}/"
    for j in range(12):
        # if j != 11: continue
        filename = f"{j}_output.py"
        program_path = os.path.join(code_path, filename)
        if not os.path.exists(program_path): continue
        score_path = f"automation_2/scores/{layer}_{j}_score.txt"
        os.makedirs(os.path.dirname(score_path), exist_ok=True)

        try:
            spec = importlib.util.spec_from_file_location(f"layer{layer}_j{j}", program_path)
            module = importlib.util.module_from_spec(spec)
            module.__dict__['np'] = np
            spec.loader.exec_module(module)
        except Exception as e:
            print(f"Layer {layer}, Head {j} - Error: {e}")
            with open(score_path, "w") as f: f.write(f"Error loading module: {e}")
            failed_programs.append((layer, j))
            scores.append(-1)
            continue

        for attr_name in dir(module):
            attr = getattr(module, attr_name)
            if isinstance(attr, types.FunctionType):
                program_to_test = attr
                break

        try:
            score = score_prediction(model, tokenizer, (layer, j), program_to_test, generic_sentences[0], distance="jsd", output=False)
            print(f"Layer {layer}, Head {j} - Score: {score:.2f}")
            with open(score_path, "w") as f: f.write(f"{score:.2f}")
            scores.append(score)
        except Exception as e:
            print(f"Layer {layer}, Head {j} - Error: {e}")
            with open(score_path, "w") as f: f.write(f"Error during scoring: {e}")
            failed_programs.append((layer, j))
            scores.append(-1)
            continue

num_scored = len([s for s in scores if s != -1 and not np.isnan(s)])
print(f"Number of Successfully Scored Heads: {num_scored} out of {len(scores)}")

avg_score = sum([s for s in scores if s != -1 and not np.isnan(s)]) / len([s for s in scores if s != -1 and not np.isnan(s)])
print(f"Average Score (excluding errors): {avg_score:.2f}")

In [ ]:
# make scores into a matrix (12,12) with -1 for failed programs

sq_score = np.full((12, 12), -1.0)
for idx, score in enumerate(scores):
    layer = idx // 12
    head = idx % 12
    sq_score[layer, head] = score

sq_score

In [ ]:
# loop through bert and bert2 scores, take best score and build sq_Score matrix

scores_1 = np.array([])
scores_2 = np.array([])
scores_min = np.array([])

for layer in range(12):
    for head in range(12):
        score1_path = os.path.join("automation_results_bert", "scores", f"layer{layer}_head{head}_score.txt")
        score2_path = os.path.join("automation_results_bert_2", "scores", f"layer{layer}_head{head}_score.txt")
        if os.path.exists(score1_path):
            with open(score1_path, "r") as f:
                try:
                    score1 = round(float(f.read().strip()), 2)
                    scores_1 = np.append(scores_1, score1)
                except:
                    score1 = 1
        # if os.path.exists(score2_path):
        try:
            with open(score2_path, "r") as f:
                try:
                    score2 = round(float(f.read().strip()), 2)
                    scores_2 = np.append(scores_2, score2)
                except:
                    score2 = 1
                    scores_2 = np.append(scores_2, score2)
        except:
            scores_2 = np.append(scores_2, 1)

        best_score = min(score1, score2)
        scores_min = np.append(scores_min, best_score)

sq_scores = scores_min.reshape((12, 12))
np.save("bert_scores_1.npy", scores_1.reshape((12, 12)))
np.save("bert_scores_2.npy", scores_2.reshape((12, 12)))
np.save("bert_headscores.npy", scores_min.reshape((12, 12)))

In [ ]:
# iterate through bert heads, check if corresponding score in automation_results_bert/scores and automation_results_bert_2/scores. save the higher score
scores = []
for layer in range(12):
    for head in range(12):
        score1_path = os.path.join("automation_results_bert", "scores", f"layer{layer}_head{head}_score.txt")
        score2_path = os.path.join("automation_results_bert_3", "scores", f"layer{layer}_head{head}_score.txt")
        score1, score2 = None, None
        if os.path.exists(score1_path):
            with open(score1_path, 'r') as f:
                score1 = float(f.read().strip())
            
        if os.path.exists(score2_path):
            with open(score2_path, 'r') as f:
                try:
                    score2 = float(f.read().strip())
                except ValueError:
                    print(f"Invalid score in {score2_path}")
        
        if score1 is not None and score2 is not None:
            # print("score1:", score1, "score2:", score2, "min:", min(score1, score2))
            scores.append(min(score1, score2))
        elif score1 is not None:
            scores.append(score1)
        elif score2 is not None:
            scores.append(score2)
        else:
            scores.append(None)

scores = np.array(scores).reshape(12, 12)

In [ ]:
# ordinary score plot

colors = "Grays"
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
masked_sq = np.ma.masked_where(sq_score == -1, sq_score)
norm = PowerNorm(gamma=1.8, vmin=sq_score.min(), vmax=sq_score.max())
cmap = plt.cm.get_cmap(colors).copy()
cmap.set_bad(color='gray')
im2 = ax.imshow(masked_sq, cmap=cmap, aspect='auto', norm=norm)
im2.set_clim(vmin=0, vmax=1)
cbar = plt.colorbar(im2, ax=ax)
ax.set_xticks(range(12))
ax.set_yticks(range(12))
ax.set_xticklabels([i for i in range(12)], rotation=90)
ax.set_yticklabels([i for i in range(12)])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
# put a space element in between automation and scores in text
title = (
    r'$\mathbf{Automation \ Scores}$'  # \mathbf makes the text bold
    '\n\nMethod: No Refinement'
    f'\nScores | {model.config.architectures[0]}'  # Example: replaced model.config...
)
plt.title(f"{title}\n")
plt.show()

In [ ]:
scores = np.load("data/k62_bert_automation_scores.npy")
sq_score = np.reshape(scores, (12, 12))

# Mask invalid entries
masked_sq = np.ma.masked_where(sq_score == -1, sq_score)

# Setup base heatmap
colors = "Greys"
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
norm = PowerNorm(gamma=1.8, vmin=sq_score.min(), vmax=sq_score.max())
cmap = plt.cm.get_cmap(colors).copy()
cmap.set_bad(color='gray')

# --- Base image ---
im = ax.imshow(masked_sq, cmap=cmap, aspect='auto', norm=norm)
im.set_clim(vmin=0, vmax=1)
cbar = plt.colorbar(im, ax=ax)

# --- Overlay red cells (< 0.4) ---
threshold_value = 0.2
highlight_mask = sq_score < threshold_value
red_overlay = np.zeros((*sq_score.shape, 4))
red_overlay[highlight_mask] = [0.9, 0, 0.2, 1]  # nice blue RGBA color: 

# Plot on top (no transparency, exact green)
ax.imshow(red_overlay, aspect='auto')

# Axes
ax.set_xticks(range(12))
ax.set_yticks(range(12))
ax.set_xticklabels(range(12), rotation=90)
ax.set_yticklabels(range(12))

# Title
title = (
    r'$\mathbf{BERT \ Automation \ Scores}$'
    f'\nHighlights for Great Scores | Colorful if score < {threshold_value}'
    '\n\nMethod: Auto (K=N)'
    f'\n{np.sum(highlight_mask)} highlighted scores ({100 * np.sum(highlight_mask) / highlight_mask.size:.1f}%) | Avg_Score: {np.mean(sq_score):.3f}'
)
plt.title(f"{title}\n")

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
colors = "Grays_r"
plt.rcParams['text.usetex'] = False

score_threshold = 0.4

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
highlighted_sq = np.where(sq_score < score_threshold, sq_score, np.nan)
# make all non-highlighted values white (1.0)
highlighted_sq = np.where(np.isnan(highlighted_sq), 1.0, highlighted_sq)
norm = PowerNorm(gamma=1.8, vmin=sq_score.min(), vmax=sq_score.max())
cmap = plt.cm.get_cmap(colors).copy()
cmap.set_bad(color='gray')
im2 = ax.imshow(highlighted_sq, cmap=cmap, aspect='auto', norm=norm)
im2.set_clim(vmin=0, vmax=1)
cbar = plt.colorbar(im2, ax=ax)
ax.set_xticks(range(12))
ax.set_yticks(range(12))
ax.set_xticklabels([i for i in range(12)], rotation=90)
ax.set_yticklabels([i for i in range(12)])
import matplotlib
print("usetex:", matplotlib.rcParams['text.usetex'])
plt.rcParams['text.usetex'] = False

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
# make automation bold
# get number of highlighted scores
num_highlighted = np.sum(sq_score < score_threshold)
title = (
    r'$\mathbf{Highlighted\ Scores}$'
    '\n\nMethod: No Refinement'
    f'\n {num_highlighted} scores < {score_threshold} ({num_highlighted/(len(sq_score)**2)*100:.0f}%) | {model.config.architectures[0]}\n')
# title = "Automation Scores\n"
plt.title(title)
plt.show()

In [ ]:
# go through every head, get score for each pattern in patterns, save the lowest ones in scores

scores = []
# bert
model = AutoModel.from_pretrained("bert-base-uncased", output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

for layer in range(model.config.num_hidden_layers):
    for head in range(model.config.num_attention_heads):
        head_scores = []
        for pattern in patterns:
            try:
                sentence_collector = []
                for sentence in sentences[:5]:
                    score = score_prediction(model, tokenizer, (layer, head), pattern, sentence, distance="jsd", output=False)
                    sentence_collector.append((sentence, score))
                head_score = np.mean([s[1] for s in sentence_collector])
                head_scores.append((pattern.__name__, head_score))

            except Exception as e:
                print(f"Error scoring Layer {layer}, Head {head} with pattern {pattern.__name__}: {e}")
                head_scores.append((pattern.__name__, float('inf')))
        
        # get pattern with lowest score
        best_pattern, best_score = min(head_scores, key=lambda x: x[1])
        scores.append((layer, head, best_pattern, best_score))
        print(f"Layer {layer}, Head {head} - Best Pattern: {best_pattern} | Score: {best_score:.2f}\n")

In [ ]:
test_sentences = sentences[135:150]
# scores = np.zeros((model.config.num_hidden_layers, model.config.num_attention_heads))
top_k = -1

# model is bert
# model = AutoModel.from_pretrained("bert-base-uncased", output_attentions=True)
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# torch_model = model
# torch_tokenizer = tokenizer 
from sklearn.linear_model import Ridge, Lasso, LinearRegression

# model is gpt-2
model = AutoModel.from_pretrained("openai-community/gpt2", output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
torch_model = model
torch_tokenizer = tokenizer

scores = np.zeros((model.config.num_hidden_layers, model.config.num_attention_heads))
num_layers = torch_model.config.num_hidden_layers
num_heads = torch_model.config.num_attention_heads

for layer in range(num_layers):
    # if layer < 8: continue
    for head in range(num_heads):
        # if layer < 8 and head < 4: continue
        print(f"Analyzing Layer {layer}, Head {head}...")
        sentence_scores = []
        for sentence in test_sentences:
            # inputs = tokenizer(sentence, return_tensors="pt")
            # outputs = model(**inputs, output_attentions=True)
            # attention = outputs.attentions[layer][0, head].detach().numpy()
            # y = attention.flatten()
            # num_tokens = len(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0]))

            toks = tokenizer([sentence], return_tensors="pt")
            len_seq = len(toks.input_ids[0])
            # assign full attention to one random index in each row
            attention = np.zeros((len_seq, len_seq))
            for i in range(len_seq):
                attention[i, np.random.randint(0, len_seq)] = 1

            # attention = np.random.rand(len_seq, len_seq)
            attention = attention / attention.sum(axis=1, keepdims=True)
            y = attention.flatten()

            X = []
            for pattern in patterns:
                try:
                    instance = pattern(sentence, tokenizer)[1].flatten()
                    # asssert that instance is a 2d array
                    if instance.ndim != 1:
                        print(f"\t\tRemoved pattern {pattern.__name__} due to incorrect output dimensions.")
                        del patterns[patterns.index(pattern)]
                        continue


                    # check if nan in instance:
                    if np.isnan(instance).any():
                        del patterns[patterns.index(pattern)]
                        print(f"\t\tRemoved pattern {pattern.__name__} due to NaN values.")
                        continue
                    X.append(instance)
                    # print(f"Tested pattern: {pattern.__name__} successfully")

                except Exception as e:
                    # print(f"Error testing pattern {pattern.__name__}: {e}")
                    print(f"\t\tRemoved pattern {pattern.__name__} due to error during testing.")
                    del patterns[patterns.index(pattern)]
                    continue

            X_n = np.array(X).T
            y = np.nan_to_num(y.flatten())

            # methods = ["lasso", "l1", "ridge", "linear"]
            # method = "lasso"

            # if method == "lasso":
            #     reg = Lasso(alpha=0.001).fit(X_n, y)
            # elif method == "l1":
            #     reg = ElasticNet(alpha=0.001, l1_ratio=1.0).fit(X_n, y)
            # elif method == "ridge":
            #     reg = Ridge(alpha=0.001).fit(X_n, y)
            # else:
            #     reg = LinearRegression().fit(X_n, y)

            # reg = Lasso(alpha=0.01).fit(X_n, y)
            # reg = Ridge(alpha=0.001).fit(X_n, y)
            reg = LinearRegression().fit(X_n, y)
            side_length = int(np.sqrt(len(y)))
            y = y.reshape((side_length, side_length))
            top_indices = np.argsort(np.abs(reg.coef_))[-top_k:]

            if top_k == 1:
                pred_atts = []
                fn_to_trys = []

                for i in range(5):
                    fn_to_try = patterns[np.argsort(np.abs(reg.coef_))[-1 - i]]
                    pred_att = fn_to_try(sentence, torch_tokenizer)[1]
                    pred_atts.append(pred_att)
                    fn_to_trys.append(fn_to_try)

                scores_five = []
                for pred_att in pred_atts:
                    jensonshannon_distances = []
                    for row_att, row_out in zip(y, pred_att):
                        jensonshannon_distances.append(np.sqrt(js_divergence(row_att, row_out)))
                    try_score = np.mean(jensonshannon_distances)
                    scores_five.append(try_score)

                fn_winner = fn_to_trys[np.argmin(scores_five)]
                pred_att = pred_atts[np.argmin(scores_five)]

                print(f"\tUsing pattern: {fn_winner.__name__}")

            elif top_k == -1:
                pred_att = reg.intercept_ + sum(coef * mat for coef, mat in zip(reg.coef_, X))
                pred_att = pred_att.reshape((side_length, side_length))

            else:
                pred_att = reg.intercept_ + sum(reg.coef_[i] * X[i] for i in top_indices)
                pred_att = pred_att.reshape((side_length, side_length))

            jensonshannon_distances = []
            for row_att, row_out in zip(y, pred_att):
                jensonshannon_distances.append(np.sqrt(js_divergence(row_att, row_out)))
            score = np.mean(jensonshannon_distances)
            sentence_scores.append(score)
        
        # scores[layer, head] = np.mean(sentence_scores)
        scores[layer, head] = np.mean(sentence_scores)
        print(f"\tScore for Layer {layer}, Head {head}: {np.mean(sentence_scores):.2f}\n")

# for each head: do linear interpolation on patterns, set pred_att to sum with top_k hypotheses based on parameter magnitude

In [ ]:
# scores nan to mean
scores = np.nan_to_num(scores, nan=np.nanmean(scores))
scores

In [ ]:
print(len(patterns))

In [ ]:
np.save("data/interpolation_experiment_gpt2_lasso_a=0.01_k=N.npy", scores)

In [ ]:
X = []
for pattern in patterns:
    try:
        instance = pattern(sentence, tokenizer)[1].flatten()
        # asssert that instance is a 2d array
        if instance.ndim != 1:
            print(f"\t\tRemoved pattern {pattern.__name__} due to incorrect output dimensions.")
            del patterns[patterns.index(pattern)]
            continue


        # check if nan in instance:
        if np.isnan(instance).any():
            del patterns[patterns.index(pattern)]
            print(f"\t\tRemoved pattern {pattern.__name__} due to NaN values.")
            continue
        X.append(instance)
        # print(instance.shape)
        # print(f"Tested pattern: {pattern.__name__} successfully")



    except Exception as e:
        # print(f"Error testing pattern {pattern.__name__}: {e}")
        print(f"\t\tRemoved pattern {pattern.__name__} due to error during testing.")
        del patterns[patterns.index(pattern)]
        continue
X


In [ ]:
# dict: pattern name -> number of lines
pattern_line_dict = {}
for pattern, num_lines in zip(patterns, number_of_lines):
    pattern_line_dict[pattern.__name__] = num_lines

In [ ]:
line_count = []
# go through scores and assign the number of lines based on the patter name
for layer, head, pattern_name, score in scores:
    num_lines = pattern_line_dict.get(pattern_name, None)
    line_count.append(num_lines)
print(len(line_count))

In [ ]:
# i only need score from scores, not head and layer
final_scores = [s[3] for s in scores]

line_count_np = np.array(line_count).flatten()
final_scores_np = np.array(final_scores).flatten()

plt.figure(figsize=(8, 6))
plt.scatter(line_count, final_scores, color='blue')
plt.xlabel("Number of Non-Comment, Non-Blank Lines\n")
plt.ylabel("Score")
# model is bert
# model = AutoModel.from_pretrained("bert-base-uncased", output_attentions=True)
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#bold second line in title
second_line = f"{model.config.architectures[0]}"
# how to bold text in only part of title
plt.title(f"Function Length vs Score | Score is from \nAuto (K=1) Refinement\n\n{second_line}\n")
plt.grid()
plt.show()

In [ ]:
# PROGRAM COMPLEXITY ANALYSIS | FUNCTION LENGTH VS SCORE

#make scatterplot, loop through functions and count number of non-comment, non-blank lines and then get scores and at the end plot all line/score pairs

folder_of_interest = "automation_results_gpt2"

complexity = []
for layer in range(12):
    for head in range(12):
        score_path = os.path.join(folder_of_interest, "scores", f"layer{layer}_head{head}_score.txt")
        # program_path = os.path.join(folder_of_interest, "llm_code", f"code_layer_{layer}", f"{head}_output.py")
        # program example: automation_results_bert\llm_code\layer0_head0_code.py
        program_path = os.path.join(folder_of_interest, "llm_code", f"layer{layer}_head{head}_code.py")
        if not os.path.exists(score_path):
            print("here")
            continue

        with open(score_path, "r") as f:
            try:
                score = float(f.read().strip())
            except:
                continue

        with open(program_path, "r") as f:
            lines = f.readlines()
            code_lines = [line for line in lines if line.strip() and not line.strip().startswith("#")]
            num_code_lines = len(code_lines)
        
        complexity.append((num_code_lines, score))
        print(f"L{layer},H{head} | Lines: {num_code_lines} | Score: {score}")

model = AutoModel.from_pretrained("gpt2", output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

plt.figure(figsize=(8, 6))
line_counts = [item[0] for item in complexity]
scores = [item[1] for item in complexity]
plt.scatter(line_counts, scores, color='blue')
plt.xlabel("Number of Non-Comment, Non-Blank Lines\n")
plt.ylabel("Score")
# model is bert
# model = AutoModel.from_pretrained("bert-base-uncased", output_attentions=True)
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#bold second line in title
second_line = f"{model.config.architectures[0]}"
# how to bold text in only part of title
plt.title(f"Function Length vs Score | Score is from \nNo Refinement\n\n{second_line}\n")
plt.grid()
plt.show()

<div style="text-align: center; font-size: 2em;">
  <hr>
    Hypothesis Structure Experiments
  <hr>
</div>

In [ ]:
programs = [next_attention, previous_attention, same_attention, punctuation_attention, dependencies,
            last_token_attention, uniform_attention, cls_attention, eos_attention, pos_alignment,
            special_token_attention, repeated_attention, noun_modifier_attention, pronoun_attention,
            single_token_attention, root_cluster_attention]
programs

In [ ]:
import networkx as nx
from programs import *

programs = [next_attention, previous_attention, same_attention, punctuation_attention, dependencies,
            last_token_attention, uniform_attention, cls_attention, eos_attention, pos_alignment,
            special_token_attention, repeated_attention, noun_modifier_attention, pronoun_attention,
            single_token_attention, root_cluster_attention]

sentence_data = sentences[:25]
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def program_similarity(att_one, att_two):
    def js_divergence(p: np.ndarray, q: np.ndarray) -> float:
        p = np.clip(p, 1e-12, 1.0)
        q = np.clip(q, 1e-12, 1.0)
        p /= p.sum()
        q /= q.sum()
        m = 0.5 * (p + q)
        return 0.5 * (np.sum(p * np.log(p / m)) + np.sum(q * np.log(q / m))) 

    jensonshannon_distances = []
    for row_att, row_out in zip(att_one, att_two):
        jensonshannon_distances.append(np.sqrt(js_divergence(row_att, row_out)))
    score = np.mean(jensonshannon_distances)
    return score

x = len(programs)
S = np.zeros((x, x))
for i in range(x):
    print(f"calculating hypothesis similarities [{i}]: {programs[i].__name__}")
    for j in range(x):
        if i < j:
            similarities = []
            program_one = programs[i]
            program_two = programs[j]

            for sentence in sentence_data:
                h1, activations_one = program_one(sentence, tokenizer)
                h2, activations_two = program_two(sentence, tokenizer)
                similarities.append(program_similarity(activations_one, activations_two))
            
            S[i, j] = np.mean(similarities)
            S[j, i] = S[i, j]

In [ ]:
S = np.load('data/similarity_matrix_auto.npy')

def group_similar_programs(programs, S, threshold=0.6):
    groups, used = [], set()
    for i in range(len(programs)):
        if i in used: continue
        group = [i]
        used.add(i)
        
        changed = True
        while changed:
            changed = False
            for group_member in group:
                for j in range(len(programs)):
                    if j not in used and S[group_member, j] < threshold:
                        group.append(j)
                        used.add(j)
                        changed = True
        groups.append([programs[idx].__name__ for idx in group])
    
    return groups

groups = group_similar_programs(programs, S, threshold=0.6)
for i, group in enumerate(groups):
    print(f"Group {i+1}: {group}")

In [ ]:
folder = "automation_results_bert"
programs = []
for layer in range(12):
    for head in range(12):
        code_path = os.path.join(folder, "llm_code", f"layer{layer}_head{head}_code.py")
        if os.path.exists(code_path):
            spec = importlib.util.spec_from_file_location(f"layer{layer}_head{head}", code_path)
            module = importlib.util.module_from_spec(spec)
            module.__dict__['np'] = np
            try:
                spec.loader.exec_module(module)
                for attr_name in dir(module):
                    attr = getattr(module, attr_name)
                    if isinstance(attr, types.FunctionType):
                        programs.append(attr)
                        break
            except Exception as e:
                print(f"Error loading program for Layer {layer}, Head {head}: {e}")
                continue

In [ ]:
S

In [ ]:
S = (S + S.T) / 2

def group_similar_programs(programs, S, threshold=0.6):
    groups, used = [], set()
    for i in range(len(programs)):
        if i in used: continue
        group = [i]
        used.add(i)
        
        changed = True
        while changed:
            changed = False
            for group_member in group:
                for j in range(len(programs)):
                    if j not in used and S[group_member, j] < threshold:
                        group.append(j)
                        used.add(j)
                        changed = True
        groups.append([programs[idx].__name__ for idx in group])
    
    return groups

groups = group_similar_programs(programs, S, threshold=0.25)
for i, group in enumerate(groups):
    if len(group) >= 1:
        print(f"Group {i+1}: {group}\n")

In [ ]:
for program in programs:
    print(program.__name__)

In [ ]:
name_to_idx = {fn.__name__: i for i, fn in enumerate(programs)}
new_order = [name_to_idx[name] for group in groups for name in group]
S_grouped = S[np.ix_(new_order, new_order)]
# S_grouped = np.load("data/similarity_matrix_auto.npy")
colors = "Blues_r"
fig, ax = plt.subplots(1, 1, figsize=(8,8))
im2 = ax.imshow(S_grouped, cmap=colors, aspect='auto')
# ax.set_axis_off()
# ax.set_xticks(range(len(programs)))
# ax.set_yticks(range(len(programs)))
# ax.set_xticklabels([p.__name__ for p in programs], rotation=90)
# ax.set_yticklabels([p.__name__ for p in programs])
# axis off
ax.set_axis_off()
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.title("Automated Programs | Similarity Matrix\n", weight='bold')
plt.show()

In [ ]:
# automated_program_similarity_analysis

import os
import importlib.util
import types
# from AutoTokenizer import AutoTokenizer
import numpy as np

folder = "automation_results_bert"
programs = []
for layer in range(12):
    for head in range(12):
        code_path = os.path.join(folder, "llm_code", f"programs-layer_{layer}", f"{head}_output.py")
        if os.path.exists(code_path):
            spec = importlib.util.spec_from_file_location(f"layer{layer}_head{head}", code_path)
            module = importlib.util.module_from_spec(spec)
            module.__dict__['np'] = np
            try:
                spec.loader.exec_module(module)
                for attr_name in dir(module):
                    attr = getattr(module, attr_name)
                    if isinstance(attr, types.FunctionType):
                        programs.append(attr)
                        break
            except Exception as e:
                print(f"Error loading program for Layer {layer}, Head {head}: {e}")
                continue

# sentence_data = sentences[:10]
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# def program_similarity(att_one, att_two):
#     def js_divergence(p: np.ndarray, q: np.ndarray) -> float:
#         p = np.clip(p, 1e-12, 1.0)
#         q = np.clip(q, 1e-12, 1.0)
#         p /= p.sum()
#         q /= q.sum()
#         m = 0.5 * (p + q)
#         return 0.5 * (np.sum(p * np.log(p / m)) + np.sum(q * np.log(q / m))) 

#     jensonshannon_distances = []
#     for row_att, row_out in zip(att_one, att_two):
#         jensonshannon_distances.append(np.sqrt(js_divergence(row_att, row_out)))
#     score = np.mean(jensonshannon_distances)
#     return score

# x = len(programs)
# S = np.zeros((x, x))
# for i in range(x):
#     print(f"calculating hypothesis similarities [{i}]: {programs[i].__name__}")
#     for j in range(x):
#         if j % 24 == 0: print(f"  inner loop {j}/{x}")
#         if i != j:
#             similarities = []
#             program_one = programs[i]
#             program_two = programs[j]

#             for sentence in sentence_data:
#                 h1, activations_one = program_one(sentence, tokenizer)
#                 h2, activations_two = program_two(sentence, tokenizer)
#                 similarities.append(program_similarity(activations_one, activations_two))
            
#             S[i, j] = np.mean(similarities)

<div style="text-align: center; font-size: 2em;">
  <hr>
    Hypothesis Replacement Experiments
  <hr>
</div>

In [ ]:
from programs import *
from transformers import BertTokenizer, BertForMaskedLM

programs = [next_attention, previous_attention, same_attention, punctuation_attention, dependencies,
            last_token_attention, uniform_attention, cls_attention, eos_attention, pos_alignment,
            special_token_attention, repeated_attention, noun_modifier_attention, pronoun_attention,
            single_token_attention, root_cluster_attention]

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
model_replace = BertForMaskedLM.from_pretrained("bert-base-uncased")

model.eval()
model_replace.eval()

In [ ]:
# helper functions for replacing heads in llms

def has_hooks(model):
    # sanity check to see if any hooks are registered
    for module in model.modules():
        if module._forward_hooks or module._backward_hooks or module._forward_pre_hooks:
            return True
    return False

def remove_all_hooks(model):
    for module in model.modules():
        module._forward_hooks = {}
        module._backward_hooks = {}
        module._forward_pre_hooks = {}
    worked = has_hooks(model)
    print(f"All hooks removed: {not worked}")
    return model

In [ ]:
def generate_hypothesis_dictionary(model, tokenizer, sentences):
    best_fit_data = pd.read_csv('data/best_fit_refinement_bert.csv') # i, j, pattern, score
    hypothesis_dict = {} # (layer, head) -> (hypothesis, score)
    for row in best_fit_data.itertuples():
        layer = row.i
        head = row.j
        hypothesis = row.Pattern
        # hypothesis_dict[(layer, head)] = hypothesis
        # attach hypothesis to dict if it's the lowest score for that head
        if (layer, head) not in hypothesis_dict:
            hypothesis_dict[(layer, head)] = (hypothesis, row.Score)
        else:
            if row.Score < hypothesis_dict[(layer, head)][1]:
                hypothesis_dict[(layer, head)] = (hypothesis, row.Score)
    
    # collect hypothesis names set and define program_executables list
    program_executables = []
    hypothesis_names = set()
    for key in hypothesis_dict: hypothesis_names.add(hypothesis_dict[key][0])

    for name in hypothesis_names:
        flag = False
        for program in programs:
            if program.__name__ == name:
                program_executables.append(program)
                flag = True
                break
        if not flag:
            print(f"Program {name} not found among available programs.")

    print(f"\nNumber of Heads with Hypotheses: {len(hypothesis_dict)}")
    print(f"Percentage of Heads with Hypotheses: {len(hypothesis_dict) / 144 * 100:.2f}%")
    print(f"Number of Program Executables: {len(program_executables)}\n")
    return hypothesis_dict, program_executables

best_fits, programs_ex = generate_hypothesis_dictionary(model, tokenizer, sentences)
print([program.__name__ for program in programs_ex])
best_fits

In [ ]:
top_percentage = 0.2
best_fits_sorted = sorted(best_fits.items(), key=lambda x: x[1][1])
best_fits_top = best_fits_sorted[:int(len(best_fits_sorted) * top_percentage)]
best_fits_top

In [ ]:
def mask_sentence(sent, tokenizer):
    tokens = tokenizer.tokenize(sent)
    if not tokens:
        return sent
    idx = np.random.randint(len(tokens))
    true_token = tokens[idx]
    tokens[idx] = tokenizer.mask_token
    return tokenizer.convert_tokens_to_string(tokens), true_token

masked_sentences = []
true_tokens = []
for i, sent in enumerate(sentences):
    masked_sent, true_token = mask_sentence(sent, tokenizer)
    if i < 6: print(f"Original: {sent}\nMasked:   {masked_sent}\n")
    masked_sentences.append(masked_sent)
    true_tokens.append(true_token)

In [ ]:
def single_perplexity(sentence, true_token):
    inputs = tokenizer(sentence, return_tensors="pt")
    mask_token_index = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)
    if len(mask_token_index[0]) == 0:
        raise ValueError(f"No [MASK] token found in: {sentence}")
    with torch.no_grad():
        logits = model(**inputs).logits  # shape: [1, seq_len, vocab_size]
    mask_pos = mask_token_index[1].item()
    masked_logits = logits[0, mask_pos, :]  # shape: [vocab_size]
    true_id = tokenizer.convert_tokens_to_ids(true_token)
    log_probs = torch.log_softmax(masked_logits, dim=-1)
    log_prob = log_probs[true_id] 
    return torch.exp(-log_prob).item()

def full_perplexity(sentence):
    tokens = tokenizer.tokenize(sentence)
    ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = torch.tensor([ids])
    log_probs = []
    for i in range(1, len(tokens)-1):  # consider all tokens except [CLS], [SEP]
        masked = input_ids.clone()
        masked[0, i] = tokenizer.mask_token_id
        with torch.no_grad():
            logits = model(masked).logits
        prob = torch.log_softmax(logits[0, i], dim=-1)[input_ids[0, i]]
        log_probs.append(prob)
    return torch.exp(-torch.stack(log_probs).mean())

average_ppl_single = []
average_ppl_full = []
for i, (sent, true_tok) in enumerate(zip(masked_sentences[:100], true_tokens[:100])):
    ppl_s = single_perplexity(sent, true_tok)
    ppl_f = full_perplexity(sent)
    average_ppl_single.append(ppl_s)
    average_ppl_full.append(ppl_f)
    if i % 50 == 0: print(f"Sentence {i}:\tPerplexity(s): {ppl_s:.3f}\tPerplexity(f): {ppl_f:.3f}")

In [ ]:
def register_hypothesis_hooks(model, best_fits, tokenizer):
    hooks = []
    def replace_activation_with_hypothesis(layer, head, sentence, tokenizer):
        hypothesis = best_fits.get((layer, head), (None, None))[0]
        program_to_use = programs_ex[[i for i, p in enumerate(programs_ex) if p.__name__ == hypothesis][0]] if hypothesis else None
        # print(f"Replacing Layer {layer}, Head {head} with Hypothesis: {hypothesis} | program: {program_to_use}")
        out = program_to_use(sentence, tokenizer) if program_to_use else None
        return out

    for (layer_idx, head_idx) in best_fits:
        target_layer = model.bert.encoder.layer[layer_idx].attention.self

        def make_hook(layer_i, head_i):
            def hook_fn(module, input, output):
                context_layer, attn_probs = output
                print(context_layer.shape)
                # B, S, H, D = context_layer.shape  # (batch, seq_len, num_heads, head_dim)

                sentence = tokenizer.decode(input[0][0], skip_special_tokens=True)
                out = replace_activation_with_hypothesis(layer_i, head_i, sentence, tokenizer)  # shape (S, S)

                # project (S, S) → (B, S, D)
                D = context_layer.size(-1)
                rand_head = out.unsqueeze(0).repeat(B, 1, 1)[:, :, :D]
                context_layer[:, :, head_i, :] = rand_head
                return (context_layer, attn_probs)
            return hook_fn

        hook = target_layer.register_forward_hook(make_hook(layer_idx, head_idx))
        hooks.append(hook)
    return hooks

hooks = register_hypothesis_hooks(
    model_replace,
    best_fits=best_fits,
    tokenizer=tokenizer
)

In [ ]:
ppl_subset_replaced = []
for i, (sentence, true_token) in tqdm(enumerate(zip(masked_sentences[:100], true_tokens[:100]))):
    inputs = tokenizer(sentence, return_tensors="pt")
    mask_token_index = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)
    if len(mask_token_index[0]) == 0:
        raise ValueError(f"No [MASK] token found in: {sentence}")
    with torch.no_grad():
        logits = model_replace(**inputs).logits  # shape: [1, seq_len, vocab_size]
    mask_pos = mask_token_index[1].item()
    masked_logits = logits[0, mask_pos, :]  # shape: [vocab_size]
    true_id = tokenizer.convert_tokens_to_ids(true_token)
    log_probs = torch.log_softmax(masked_logits, dim=-1)
    log_prob = log_probs[true_id] 
    # if i % 10 == 0: print(torch.exp(-log_prob).item())
    ppl_subset_replaced.append(torch.exp(-log_prob).item())

ppl_subset_normal = []
for i, (sentence, true_token) in tqdm(enumerate(zip(masked_sentences[:100], true_tokens[:100]))):
    inputs = tokenizer(sentence, return_tensors="pt")
    mask_token_index = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)
    if len(mask_token_index[0]) == 0:
        raise ValueError(f"No [MASK] token found in: {sentence}")
    with torch.no_grad():
        logits = model(**inputs).logits  # shape: [1, seq_len, vocab_size]
    mask_pos = mask_token_index[1].item()
    masked_logits = logits[0, mask_pos, :]  # shape: [vocab_size]
    true_id = tokenizer.convert_tokens_to_ids(true_token)
    log_probs = torch.log_softmax(masked_logits, dim=-1)
    log_prob = log_probs[true_id] 
    # if i % 10 == 0: print(torch.exp(-log_prob).item())
    ppl_subset_normal.append(torch.exp(-log_prob).item())

print(f"Average Perplexity disabled: pd-describe{(pd.Series(ppl_subset_replaced).describe())}\n")
print(f"Average Perplexity normal: pd-describe{(pd.Series(ppl_subset_normal).describe())}")

In [ ]:
print(sentences[3])
# tokenizer.decode(tokenizer(sentences[0])['input_ids'], skip_special_tokens=True)
tokens = tokenizer.tokenize(sentences[0])
input = tokenizer(sentences[0], return_tensors="pt")
print(len(input), input['input_ids'].shape[1])

decoded = tokenizer.decode(input['input_ids'][0], skip_special_tokens=True)
_, activation_hypothesis = get_hypothesis_head(layer_id, head, decoded, tokenizer)
print(activation_hypothesis.shape)

In [ ]:
def get_hypothesis_head(layer, head, sentence, tokenizer):
        hypothesis = best_fits.get((layer, head), (None, None))[0]
        program_to_use = programs_ex[[i for i, p in enumerate(programs_ex) if p.__name__ == hypothesis][0]] if hypothesis else None
        # print(f"Replacing Layer {layer}, Head {head} with Hypothesis: {hypothesis} | program: {program_to_use}")
        out = program_to_use(sentence, tokenizer) if program_to_use else None
        return out

def replace_attention_head(module, input, output):
    """ module: BertSelfAttention | output: tuple (context_layer, attention_probs)"""
    context_layer, attn_probs = output
    batch, seq_len, hidden_dim = context_layer.shape
    num_heads = module.num_attention_heads
    head_dim = hidden_dim // num_heads
    context_layer = context_layer.view(batch, seq_len, num_heads, head_dim)

    # print current sentence
    # print(f"Current Sentence: {inputs['input_ids']}")
    # decoded_sentence = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)
    # print(f"Decoded Sentence: {decoded_sentence}")
    # decoded_sentence = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)
    # decoded_sentences = tokenizer.batch_decode(inputs["input_ids"], skip_special_tokens=True)

    # for decoded_sentence in decoded_sentences:
        # print(f"Decoded Sentence: {decoded_sentence}")

    # layer_id = getattr(module, "layer_id", None)
    # heads = []
    # for (layer, head), (fn_name, score) in best_fits_top:
        # if layer == layer_id:
            # print(layer, head, fn_name)
            # heads.append(head)

    # zero_out = 10 # percent of all heads
    # random_heads = np.random.choice(range(num_heads), size=int(num_heads * zero_out / 100), replace=False)
    # random number from 1 to 100 (2 random numbers in a list)
    random_heads = np.random.choice(range(num_heads), size=6, replace=False)
    print(len(random_heads))
    for i in random_heads:
        context_layer[:, :, i, :] = 0

    # for head in heads:
        # activation_hypothesis = np.eye(seq_len)
        # _, activation_hypothesis = get_hypothesis_head(layer_id, head, decoded_sentence, tokenizer)
        # assert activation_hypothesis.shape == (seq_len, seq_len), f"Hypothesis shape {activation_hypothesis.shape} does not match expected {(seq_len, seq_len)} for '{decoded_sentence}'"

        # broadcasted_hypothesis = torch.matmul(
        #     torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
        #     context_layer[:, :, head, :]
        # )
        # context_layer[:, :, head, :] = broadcasted_hypothesis

    # if layer_id == 0:
    #     head = 6
    #     # uniform attention np
    #     activation_hypothesis = np.ones((seq_len, seq_len))
    #     activation_hypothesis = activation_hypothesis / activation_hypothesis.sum(axis=1, keepdims=True)
        
    #     broadcasted_hypothesis = torch.matmul(
    #         torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #         context_layer[:, :, head, :]
    #     )
    #     context_layer[:, :, head, :] = broadcasted_hypothesis

    #     head = 4
    #     # uniform attention np
    #     activation_hypothesis = np.ones((seq_len, seq_len))
    #     activation_hypothesis = activation_hypothesis / activation_hypothesis.sum(axis=1, keepdims=True)
    #     broadcasted_hypothesis = torch.matmul(
    #         torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #         context_layer[:, :, head, :]
    #     )
    #     context_layer[:, :, head, :] = broadcasted_hypothesis

    #     head = 0
    #     # uniform attention np
    #     activation_hypothesis = np.ones((seq_len, seq_len)) 
    #     activation_hypothesis = activation_hypothesis / activation_hypothesis.sum(axis=1, keepdims=True)
    #     broadcasted_hypothesis = torch.matmul(
    #         torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #         context_layer[:, :, head, :]
    #     )
    #     context_layer[:, :, head, :] = broadcasted_hypothesis

    # if layer_id == 1:
    #     head = 6
    #     # cls attention np
    #     activation_hypothesis = np.zeros((seq_len, seq_len))
    #     activation_hypothesis[:, 0] = 1.0

    #     broadcasted_hypothesis = torch.matmul(
    #         torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #         context_layer[:, :, head, :]
    #     )
    #     context_layer[:, :, head, :] = broadcasted_hypothesis

    #     head = 9
    #     # relative position attention np
    #     activation_hypothesis = np.zeros((seq_len, seq_len))
    #     for i in range(seq_len):
    #         for j in range(seq_len):
    #             if j > i:
    #                 activation_hypothesis[i, j] = 1.0
    #     activation_hypothesis = activation_hypothesis / activation_hypothesis.sum(axis=1, keepdims=True)
    #     broadcasted_hypothesis = torch.matmul(
    #         torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #         context_layer[:, :, head, :]
    #     )
    #     context_layer[:, :, head, :] = broadcasted_hypothesis

    # if layer_id == 2:
    #     # next attention np
    #     activation_hypothesis = np.zeros((seq_len, seq_len))
    #     activation_hypothesis[0, 1:] = 1.0
    #     # activation_hypothesis = activation_hypothesis / activation_hypothesis.sum(axis=1, keepdims=True)
        
    #     head = 0
    #     broadcasted_hypothesis = torch.matmul(
    #         torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #         context_layer[:, :, head, :]
    #     )
    #     context_layer[:, :, head, :] = broadcasted_hypothesis

    #     head = 9
    #     broadcasted_hypothesis = torch.matmul(
    #         torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #         context_layer[:, :, head, :]
    #     )
    #     context_layer[:, :, head, :] = broadcasted_hypothesis
        
    # if layer_id == 7:
    #     head = 6
    #     # eos attention np
    #     activation_hypothesis = np.zeros((seq_len, seq_len))
    #     activation_hypothesis[:, -1] = 1.0

    #     broadcasted_hypothesis = torch.matmul(
    #         torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #         context_layer[:, :, head, :]
    #     )
    #     context_layer[:, :, head, :] = broadcasted_hypothesis

    # if layer_id == 8:
    #     head = 2
    #     # eos attention np
    #     activation_hypothesis = np.zeros((seq_len, seq_len))
    #     activation_hypothesis[:, -1] = 1.0

    #     broadcasted_hypothesis = torch.matmul(
    #         torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #         context_layer[:, :, head, :]
    #     )
    #     context_layer[:, :, head, :] = broadcasted_hypothesis

    # [((2, 0), ('next_attention', 0.0134178736883759)),
    # ((2, 9), ('next_attention', 0.0195240472425213)),
    # ((0, 0), ('uniform_attention', 0.1473364017258112)),
    # ((1, 6), ('cls_attention', 0.1820107218156672)),
    # ((8, 2), ('eos_attention', 0.198991498540351)),
    # ((0, 4), ('uniform_attention', 0.2081918466789174)),
    # ((1, 9), ('relative_position_attention', 0.2308144694156767)),
    # ((7, 6), ('eos_attention', 0.2383215559745129)),
    # ((0, 6), ('uniform_attention', 0.246593192497212))]

    context_layer = context_layer.reshape(batch, seq_len, hidden_dim)
    return (context_layer, attn_probs)

# target_layer = model_replace.bert.encoder.layer[2].attention.self
# hook = target_layer.register_forward_hook(replace_attention_head)

for layer_number, layer in enumerate(model_replace.bert.encoder.layer):
    layer.attention.self.layer_id = layer_number
    if layer_number in [2]:
        layer.attention.self.register_forward_hook(replace_attention_head)

ppl_zero_50 = []
for i, (sentence, true_token) in tqdm(enumerate(zip(masked_sentences[:400], true_tokens[:400]))):
    inputs = tokenizer(sentence, return_tensors="pt")
    mask_token_index = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)
    if len(mask_token_index[0]) == 0:
        raise ValueError(f"No [MASK] token found in: {sentence}")
    with torch.no_grad():
        logits = model_replace(**inputs).logits  # shape: [1, seq_len, vocab_size]

    # calculate full psuedo-perplexity
    mask_pos = mask_token_index[1].item()
    masked_logits = logits[0, mask_pos, :]  # shape: [vocab_size]
    true_id = tokenizer.convert_tokens_to_ids(true_token)
    log_probs = torch.log_softmax(masked_logits, dim=-1)
    log_prob = log_probs[true_id] 
    # if i % 10 == 0: print(torch.exp(-log_prob).item())
    ppl_zero_50.append(torch.exp(-log_prob).item())

# ppl_subset_normal = []
# for i, (sentence, true_token) in tqdm(enumerate(zip(masked_sentences[:400], true_tokens[:400]))):
#     inputs = tokenizer(sentence, return_tensors="pt")
#     mask_token_index = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)

#     if len(mask_token_index[0]) == 0:
#         raise ValueError(f"No [MASK] token found in: {sentence}")
#     with torch.no_grad():
#         logits = model(**inputs).logits  # shape: [1, seq_len, vocab_size]
#     mask_pos = mask_token_index[1].item()
#     masked_logits = logits[0, mask_pos, :]  # shape: [vocab_size]
#     true_id = tokenizer.convert_tokens_to_ids(true_token)
#     log_probs = torch.log_softmax(masked_logits, dim=-1)
#     log_prob = log_probs[true_id] 
#     # if i % 10 == 0: print(torch.exp(-log_prob).item())
#     ppl_subset_normal.append(torch.exp(-log_prob).item())

# print('\n')
print('\nWeird: ', np.median(ppl_subset_replaced), '\t', np.mean(ppl_subset_replaced))
print('Normal: ', np.median(ppl_subset_normal), '\t', np.mean(ppl_subset_normal), '\n')

print(f"Average Perplexity weird: pd-describe{(pd.Series(ppl_subset_replaced).describe())}\n")
print(f"Average Perplexity normal: pd-describe{(pd.Series(ppl_subset_normal).describe())}")

In [ ]:
# boxplots:
import matplotlib.pyplot as plt
data = []
for dataset in [ppl_subset_normal, ppl_zero_10, ppl_zero_20, ppl_zero_30]:
    threshold = np.percentile(dataset, 92)
    filtered_data = [x for x in dataset if x <= threshold]
    data.append(filtered_data)
normal_perplexity = f'Normal Perplexity\nMedian={np.median(data[0]):.2f}\nMean={np.mean(data[0]):.2f}\nStd={np.std(data[0]):.2f}'
strange_perplexity_10 = f'Zero Perplexity 10%\nMedian={np.median(data[1]):.2f}\nMean={np.mean(data[1]):.2f}\nStd={np.std(data[1]):.2f}'
strange_perplexity_20 = f'Zero Perplexity 20%\nMedian={np.median(data[2]):.2f}\nMean={np.mean(data[2]):.2f}\nStd={np.std(data[2]):.2f}'
strange_perplexity_30 = f'Zero Perplexity 30%\nMedian={np.median(data[3]):.2f}\nMean={np.mean(data[3]):.2f}\nStd={np.std(data[3]):.2f}'

labels = [normal_perplexity, strange_perplexity_10, strange_perplexity_20, strange_perplexity_30]
plt.figure(figsize=(8, 6))
plt.title("Perplexity Comparison | Zeroed Out Heads vs Normal Heads\n", weight='bold')
plt.boxplot(data, labels=labels)

In [ ]:
# boxplots:
import matplotlib.pyplot as plt
data = []
for dataset in [ppl_subset_normal, ppl_weird]:
    threshold = np.percentile(dataset, 95)
    filtered_data = [x for x in dataset if x <= threshold]
    data.append(filtered_data)
normal_perplexity = f'Normal Perplexity\nMedian={np.median(data[0]):.2f}\nMean={np.mean(data[0]):.2f}\nStd={np.std(data[0]):.2f}'
strange_perplexity = f'Replacement Perplexity\nMedian={np.median(data[1]):.2f}\nMean={np.mean(data[1]):.2f}\nStd={np.std(data[1]):.2f}'
labels = [normal_perplexity, strange_perplexity]
plt.figure(figsize=(8, 6))
plt.title("Perplexity Comparison | Hypothesis-Replaced Heads vs Normal Heads\n", weight='bold')
plt.boxplot(data, labels=labels)

In [ ]:
np.median(ppl_weird)

In [ ]:
arr = [1,2,3,4,5,6,7,8,9]
# use pd describe to show statistics
import pandas as pd
print(pd.Series(arr).describe())

In [ ]:
model_replace = BertForMaskedLM.from_pretrained("bert-base-uncased")
model_replace.load_state_dict(model.state_dict(), strict=True)
model_replace.eval()

In [ ]:
model_replace = remove_all_hooks(model_replace)

In [ ]:
print("Model has hooks: model_replace", has_hooks(model_replace))
print("Model has hooks: model", has_hooks(model))

In [ ]:
def handle_outliers(data):
    iqr = False
    if iqr:
        q1 = np.percentile(data, 25)
        q3 = np.percentile(data, 75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        return [x for x in data if lower_bound <= x <= upper_bound]
    else:
        return data[:-20]

average_ppl_single_out = handle_outliers(average_ppl_single)
average_ppl_full_out = handle_outliers(average_ppl_full)

single = f'Single Token Perplexity\nMedian={np.median(average_ppl_single):.2f}\nMean={np.mean(average_ppl_single):.2f}\nStd={np.std(average_ppl_single):.2f}'
all_tokens = f'All Token Perplexity\nMedian={np.median(average_ppl_full):.2f}\nMean={np.mean(average_ppl_full):.2f}\nStd={np.std(average_ppl_full):.2f}'

plt.figure(figsize=(8, 6))
plt.boxplot([average_ppl_single_out, average_ppl_full_out, average_ppl_full_out, average_ppl_full_out], labels=[single, all_tokens, all_tokens, all_tokens])
plt.title(f'Perplexity Comparison: BERT-base\nDataset: Tiny Sentences | len={len(average_ppl_full)}', weight='bold')
plt.grid(axis='y')
plt.show()

In [ ]:
sentence_subset = sentences[:1_000]

def replace_activation_with_hypothesis(layer, head, sentence, model, tokenizer):
    hypothesis = best_fits.get((layer, head), (None, None))[0]
    program_to_use = programs_ex[[i for i, p in enumerate(programs_ex) if p.__name__ == hypothesis][0]] if hypothesis else None

    print(f"Replacing Layer {layer}, Head {head} with Hypothesis: {hypothesis} | program: {program_to_use}")
    out = program_to_use(sentence, tokenizer) if program_to_use else None
    return out

def make_replacement_hook(layer_idx, head_idx):
    """
    Returns a hook function that replaces a specific head's activations.
    """
    def hook(module, input, output):
        # output shape: (batch_size, num_heads, seq_length, head_dim)
        batch_size, num_heads, seq_length, head_dim = output.shape
        for b in range(batch_size):
            sentence = tokenizer.decode(input[0][b])
            replacement = replace_activation_with_hypothesis(layer_idx, head_idx, sentence, model, tokenizer)
            if replacement is not None:
                output[b, head_idx] = torch.tensor(replacement, dtype=output.dtype)
        return output
    return hook
# 
def mask_sentence(sent, tokenizer):
    tokens = tokenizer.tokenize(sent)
    if not tokens:
        return sent
    idx = np.random.randint(len(tokens))
    tokens[idx] = tokenizer.mask_token
    return tokenizer.convert_tokens_to_string(tokens)

hooks = []
for (layer, head), (hypothesis, score) in best_fits_top:
    hook = make_replacement_hook(layer, head)
    hooks.append(hook)

# evaluate bertbase model on masking with no hooks and calculate perplexity
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import CrossEntropyLoss
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

model.eval()
loss_fct = CrossEntropyLoss()
perplexities = []
for i, sent in enumerate(sentence_subset):
    masked = mask_sentence(sent, tokenizer)
    inputs = tokenizer(masked, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    logits = logits.view(-1, model.config.vocab_size)
    perplexity = torch.exp(loss_fct(logits, inputs["input_ids"].view(-1)))
    perplexities.append(perplexity.item())
    if i % 20 == 0: print(f"{i}/{len(sentence_subset)}: Perplexity without Hooks: {perplexity.item():.2f}")
print(f"Average Perplexity without Hooks: {np.mean(perplexities):.2f}")

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import CrossEntropyLoss
loss_fct = CrossEntropyLoss(ignore_index=-100)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
model.eval()

def compute_bert_cross_entropy(sentence, model, tokenizer, replacement=False):
    tokens = tokenizer.tokenize(sentence)
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    n_tokens = len(tokens)
    losses = []

    for i in range(n_tokens):
        masked_input_ids = input_ids.copy()
        masked_input_ids[i] = tokenizer.mask_token_id

        inputs = torch.tensor([masked_input_ids])
        labels = torch.tensor([input_ids])
        labels[0, :i] = -100
        labels[0, i+1:] = -100

        with torch.no_grad():
            if replacement:
                
            else:
                outputs = model(inputs, labels=labels)
                loss = outputs.loss
                losses.append(loss.item())

    return np.mean(losses)

sentence_subset = sentences[:50]
cross_entropies = []

for i, sent in enumerate(sentence_subset):
    ce_loss = compute_bert_cross_entropy(sent, model, tokenizer)
    cross_entropies.append(ce_loss)
    if i % 2 == 0:
        print(f"{i}/{len(sentence_subset)}: Cross-Entropy = {ce_loss:.4f}")

print(f"Average Cross-Entropy: {np.mean(cross_entropies):.4f}")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
# from nnsight import InstrumentedModel
from nnsight import LanguageModel
import math
import random
from tqdm import tqdm

# ------------------------------------------------------------
# 1. Setup
# ------------------------------------------------------------
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# We'll use the MLM head so logits are token-prediction probabilities
hf_model = AutoModelForMaskedLM.from_pretrained(model_name)
model = LanguageModel(hf_model)   # wrap for nnsight tracing
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
hf_model.to(device)

# Heads you plan to replace
replace_heads = [(0, 0), (4, 4), (11, 11)]

# ------------------------------------------------------------
# 2. Prepare evaluation sentences
# ------------------------------------------------------------
sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "Artificial intelligence is transforming many industries.",
    "The capital of France is Paris.",
    "BERT models are powerful for natural language understanding.",
    "Machine learning can detect patterns in data."
]

# Simple masking utility: mask one random non-special token per sentence
def mask_sentence(sent, tokenizer):
    tokens = tokenizer.tokenize(sent)
    if not tokens:
        return sent
    idx = random.randrange(len(tokens))
    tokens[idx] = tokenizer.mask_token
    return tokenizer.convert_tokens_to_string(tokens)

masked_sentences = [mask_sentence(s, tokenizer) for s in sentences]

# ------------------------------------------------------------
# 3. Utility: compute MLM loss/perplexity for a batch
# ------------------------------------------------------------
def compute_mlm_loss(model, tokenizer, sentences, device):
    """Return total cross-entropy loss and perplexity for a list of masked sentences."""
    model.eval()
    total_loss = 0.0
    total_tokens = 0

    for sent in sentences:
        inputs = tokenizer(sent, return_tensors="pt").to(device)
        # find masked index
        mask_index = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)
        if len(mask_index[0]) == 0:
            continue
        labels = inputs["input_ids"].clone()

        with torch.no_grad():
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss
        total_loss += loss.item()
        total_tokens += 1

    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)
    return avg_loss, perplexity

# ------------------------------------------------------------
# 4. Evaluate baseline (no head replacement)
# ------------------------------------------------------------
print("Evaluating baseline BERT ...")
baseline_loss, baseline_ppl = compute_mlm_loss(hf_model, tokenizer, masked_sentences, device)
print(f"Baseline cross-entropy: {baseline_loss:.4f}, Perplexity: {baseline_ppl:.4f}")

# ------------------------------------------------------------
# 5. Evaluate with head replacements
# ------------------------------------------------------------
print("\nEvaluating with replaced heads ...")

total_loss_replaced = 0.0
total_tokens = 0

for sent in tqdm(masked_sentences):
    inputs = tokenizer(sent, return_tensors="pt").to(device)
    labels = inputs["input_ids"].clone()

    # Run a traced forward pass where we inject our hypotheses
    with model.trace(inputs) as trace:
        # Get seq_len and batch size
        seq_len = trace["input_ids"].shape[1]
        batch_size = trace["input_ids"].shape[0]

        for (layer_i, head_j) in replace_heads:
            attn_path = f"bert.encoder.layer.{layer_i}.attention.self.softmax"
            attn_probs = trace[attn_path]  # shape [batch, num_heads, seq_len, seq_len]

            # Obtain your hypothesis n×n matrix
            hypothesis = hypothesize_attention(layer_i, head_j).to(device)

            # Ensure shapes match (pad/trim if necessary)
            if hypothesis.shape != (seq_len, seq_len):
                raise ValueError(f"Hypothesis shape {hypothesis.shape} != ({seq_len}, {seq_len})")
            attn_probs[:, head_j, :, :] = hypothesis

        outputs = trace.output  # forward pass continues with replaced attention

    # Compute MLM loss for this sentence
    logits = outputs.logits
    loss_fn = torch.nn.CrossEntropyLoss()
    loss = loss_fn(logits.view(-1, tokenizer.vocab_size), labels.view(-1))
    total_loss_replaced += loss.item()
    total_tokens += 1

avg_loss_replaced = total_loss_replaced / total_tokens
ppl_replaced = math.exp(avg_loss_replaced)

# ------------------------------------------------------------
# 6. Compare results
# ------------------------------------------------------------
print("\n=== RESULTS ===")
print(f"Baseline Cross-Entropy: {baseline_loss:.4f}")
print(f"Replaced  Cross-Entropy: {avg_loss_replaced:.4f}")
print(f"Δ Cross-Entropy: {avg_loss_replaced - baseline_loss:+.4f}")
print(f"Baseline Perplexity: {baseline_ppl:.4f}")
print(f"Replaced  Perplexity: {ppl_replaced:.4f}")
print(f"Δ Perplexity: {ppl_replaced - baseline_ppl:+.4f}")


In [ ]:
import matplotlib.pyplot as plt

# Data for the bar chart
scores = [5.6, 0.0]
labels = ['baseline-BERT', 'replaced-BERT']

# Create the bar chart
plt.figure(figsize=(7, 5))
plt.bar(labels, scores, color=['blue', 'salmon'])
plt.ylabel('Score')
plt.title('BERT Score Comparison (Bar Chart)')
plt.ylim(0, 6) # Ensure the y-axis starts at 0
plt.savefig('bert_score_comparison_bar_chart.png')

In [ ]:
def get_hypothesis_head(layer, head, sentence, tokenizer):
        hypothesis = best_fits.get((layer, head), (None, None))[0]
        program_to_use = programs_ex[[i for i, p in enumerate(programs_ex) if p.__name__ == hypothesis][0]] if hypothesis else None
        # print(f"Replacing Layer {layer}, Head {head} with Hypothesis: {hypothesis} | program: {program_to_use}")
        out = program_to_use(sentence, tokenizer) if program_to_use else None
        return out

def replace_attention_head(module, input, output):
    """ module: BertSelfAttention | output: tuple (context_layer, attention_probs)"""
    context_layer, attn_probs = output
    batch, seq_len, hidden_dim = context_layer.shape
    num_heads = module.num_attention_heads
    head_dim = hidden_dim // num_heads
    context_layer = context_layer.view(batch, seq_len, num_heads, head_dim)

    # print current sentence
    # print(f"Current Sentence: {inputs['input_ids']}")
    # decoded_sentence = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)
    # print(f"Decoded Sentence: {decoded_sentence}")
    # decoded_sentence = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)
    # decoded_sentences = tokenizer.batch_decode(inputs["input_ids"], skip_special_tokens=True)

    # for decoded_sentence in decoded_sentences:
        # print(f"Decoded Sentence: {decoded_sentence}")

    layer_id = getattr(module, "layer_id", None)
    # heads = []
    # for (layer, head), (fn_name, score) in best_fits_top:
        # if layer == layer_id:
            # print(layer, head, fn_name)
            # heads.append(head)

    # ((5, 1), ('eos_attention', 0.2554249071807607)),
    # ((1, 8), ('relative_position_attention', 0.2565078611780894)),
    # ((4, 3), ('pronoun_reference', 0.2626135723148083)),
    # ((7, 7), ('eos_attention', 0.2630709228109905)),
    # ((8, 6), ('eos_attention', 0.2714455279356093)),
    # ((3, 1), ('relative_position_attention', 0.2823563840604032)),
    # ((3, 7), ('relative_position_attention', 0.2825369197971396)),
    # ((7, 3), ('eos_attention', 0.2884066405333515)),
    # ((2, 3), ('relative_position_attention', 0.2897239912937058)),
    # ((2, 5), ('relative_position_attention', 0.2932481224141057))]

    # for layer in range(12):
    #     if layer in best_fits_top[0] and layer_id == layer:
    
    # if layer_id == 0:
    #     head = 6
    #     # uniform attention np
    #     activation_hypothesis = np.ones((seq_len, seq_len))
    #     activation_hypothesis = activation_hypothesis / activation_hypothesis.sum(axis=1, keepdims=True)
        
    #     broadcasted_hypothesis = torch.matmul(
    #         torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #         context_layer[:, :, head, :]
    #     )
    #     context_layer[:, :, head, :] = broadcasted_hypothesis

    #     head = 4
    #     # uniform attention np
    #     activation_hypothesis = np.ones((seq_len, seq_len))
    #     activation_hypothesis = activation_hypothesis / activation_hypothesis.sum(axis=1, keepdims=True)
    #     broadcasted_hypothesis = torch.matmul(
    #         torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #         context_layer[:, :, head, :]
    #     )
    #     context_layer[:, :, head, :] = broadcasted_hypothesis

    #     head = 0
    #     # uniform attention np
    #     activation_hypothesis = np.ones((seq_len, seq_len)) 
    #     activation_hypothesis = activation_hypothesis / activation_hypothesis.sum(axis=1, keepdims=True)
    #     broadcasted_hypothesis = torch.matmul(
    #         torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #         context_layer[:, :, head, :]
    #     )
    #     context_layer[:, :, head, :] = broadcasted_hypothesis

    # if layer_id == 1:
    #     head = 6
    #     # cls attention np
    #     activation_hypothesis = np.zeros((seq_len, seq_len))
    #     activation_hypothesis[:, 0] = 1.0

    #     broadcasted_hypothesis = torch.matmul(
    #         torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #         context_layer[:, :, head, :]
    #     )
    #     context_layer[:, :, head, :] = broadcasted_hypothesis

    #     # head = 9
    #     # # relative position attention np
    #     # activation_hypothesis = np.zeros((seq_len, seq_len))
    #     # for i in range(seq_len):
    #     #     for j in range(seq_len):
    #     #         if j > i:
    #     #             activation_hypothesis[i, j] = 1.0
    #     # activation_hypothesis = activation_hypothesis / activation_hypothesis.sum(axis=1, keepdims=True)
    #     # broadcasted_hypothesis = torch.matmul(
    #     #     torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #     #     context_layer[:, :, head, :]
    #     # )
    #     # context_layer[:, :, head, :] = broadcasted_hypothesis

    # if layer_id == 2:
    #     # next attention np
    #     activation_hypothesis = np.zeros((seq_len, seq_len))
    #     activation_hypothesis[0, 1:] = 1.0
    #     # activation_hypothesis = activation_hypothesis / activation_hypothesis.sum(axis=1, keepdims=True)
        
    #     head = 0
    #     broadcasted_hypothesis = torch.matmul(
    #         torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #         context_layer[:, :, head, :]
    #     )
    #     context_layer[:, :, head, :] = broadcasted_hypothesis

    #     head = 9
    #     broadcasted_hypothesis = torch.matmul(
    #         torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #         context_layer[:, :, head, :]
    #     )
    #     context_layer[:, :, head, :] = broadcasted_hypothesis
        
    # if layer_id == 7:
    #     head = 6
    #     # eos attention np
    #     activation_hypothesis = np.zeros((seq_len, seq_len))
    #     activation_hypothesis[:, -1] = 1.0

    #     broadcasted_hypothesis = torch.matmul(
    #         torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #         context_layer[:, :, head, :]
    #     )
    #     context_layer[:, :, head, :] = broadcasted_hypothesis

    # if layer_id == 8:
    #     head = 2
    #     # eos attention np
    #     activation_hypothesis = np.zeros((seq_len, seq_len))
    #     activation_hypothesis[:, -1] = 1.0

    #     broadcasted_hypothesis = torch.matmul(
    #         torch.from_numpy(activation_hypothesis).to(context_layer.device, context_layer.dtype),
    #         context_layer[:, :, head, :]
    #     )
    #     context_layer[:, :, head, :] = broadcasted_hypothesis

    # zero_out = 10 # percent of all heads
    # random_heads = np.random.choice(range(num_heads), size=int(num_heads * zero_out / 100), replace=False)
    # random number from 1 to 100 (2 random numbers in a list)
    random_heads = np.random.choice(range(num_heads), size=4, replace=False)
    # print(len(random_heads))
    for i in random_heads:
        context_layer[:, :, i, :] = 0

    context_layer = context_layer.reshape(batch, seq_len, hidden_dim)
    return (context_layer, attn_probs)

# target_layer = model_replace.bert.encoder.layer[2].attention.self
# hook = target_layer.register_forward_hook(replace_attention_head)

for layer_number, layer in enumerate(model_replace.bert.encoder.layer):
    layer.attention.self.layer_id = layer_number
    # if layer_number in [2]:
    layer.attention.self.register_forward_hook(replace_attention_head)

ppl_weird = []
save_tokens = []
for i, (sentence, true_token) in tqdm(enumerate(zip(sentences[:100], true_tokens[:100]))):
    # calculate full psuedo-perplexity
    # inputs = tokenizer(sentence, return_tensors="pt")
    # mask_token_index = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)
    # if len(mask_token_index[0]) == 0:
    #     raise ValueError(f"No [MASK] token found in: {sentence}")
    # with torch.no_grad():
    #     logits = model_replace(**inputs).logits  # shape: [1, seq_len, vocab_size]
    # tokens = tokenizer.tokenize(sentence)
    # ids = tokenizer.convert_tokens_to_ids(tokens)   
    # input_ids = torch.tensor([ids])
    # log_probs = []
    # for i in range(1, len(tokens)-1):  # consider all tokens except [CLS], [SEP]
    #     masked = input_ids.clone()
    #     masked[0, i] = tokenizer.mask_token_id
    #     with torch.no_grad():
    #         logits = model_replace(masked).logits
    #     prob = torch.log_softmax(logits[0, i], dim=-1)[input_ids[0, i]]
    #     log_probs.append(prob)
    # ppl_weird.append(torch.exp(-torch.stack(log_probs).mean()).item())

    # calculate single token perplexity
    enc = tokenizer(sentence, return_tensors="pt", add_special_tokens=True)
    input_ids = enc["input_ids"][0]
    attention_mask = enc["attention_mask"]
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    # for token in tokens:
    #     save_tokens.append(token)
    per_token_ppl = []

    # Loop over tokens EXCLUDING CLS (0) and SEP (last)
    for i in range(1, len(input_ids) - 1):
        original_id = input_ids[i].item()
        original_token = tokens[i]
        masked_ids = input_ids.clone()
        masked_ids[i] = tokenizer.mask_token_id
        with torch.no_grad():
            logits = model_replace(
                masked_ids.unsqueeze(0),
                attention_mask=attention_mask).logits
        log_probs = torch.log_softmax(logits[0, i], dim=-1)
        log_prob_original = log_probs[original_id]
        ppl = torch.exp(-log_prob_original).item()
        
        # if original_token != "[MASK]": 
        ppl_weird.append(ppl)
        save_tokens.append(original_token)

# ppl_subset_normal = []
# for i, (sentence, true_token) in tqdm(enumerate(zip(masked_sentences[:100], true_tokens[:100]))):
#     inputs = tokenizer(sentence, return_tensors="pt")
#     mask_token_index = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)

#     if len(mask_token_index[0]) == 0:
#         raise ValueError(f"No [MASK] token found in: {sentence}")
#     with torch.no_grad():
#         logits = model(**inputs).logits  # shape: [1, seq_len, vocab_size]
    
#     # calculate full psuedo-perplexity
#     tokens = tokenizer.tokenize(sentence)
#     ids = tokenizer.convert_tokens_to_ids(tokens)
#     input_ids = torch.tensor([ids])
#     log_probs = []
#     for i in range(1, len(tokens)-1):  # consider all tokens except [CLS], [SEP]
#         masked = input_ids.clone()
#         masked[0, i] = tokenizer.mask_token_id
#         with torch.no_grad():
#             logits = model(masked).logits
#         prob = torch.log_softmax(logits[0, i], dim=-1)[input_ids[0, i]]
#         log_probs.append(prob)
#     ppl_subset_normal.append(torch.exp(-torch.stack(log_probs).mean()).item())

# print('\n')
# get rid of top 5% outliers
# ppl_weird = np.percentile(sorted(ppl_weird), 95)

print('\nWeird: ', np.median(ppl_weird), '\t', np.mean(ppl_weird), '\t', np.std(ppl_weird))
print('Normal: ', np.median(ppl_subset_normal), '\t', np.mean(ppl_subset_normal), '\n')

print(f"Average Perplexity weird: pd-describe{(pd.Series(ppl_weird).describe())}\n")
print(f"Average Perplexity normal: pd-describe{(pd.Series(ppl_subset_normal).describe())}")

In [ ]:
model_replace = BertForMaskedLM.from_pretrained("bert-base-uncased")
model_replace.load_state_dict(model.state_dict(), strict=True)
model_replace.eval()

In [ ]:
# Normal:  1.806810438632965 	 7845.270243987441 	 74461.00454762368
# Weird

In [ ]:
input_ids = enc["input_ids"][0]
attention_mask = enc["attention_mask"]
tokens = tokenizer.convert_ids_to_tokens(input_ids)
per_token_ppl = []

# Loop over tokens EXCLUDING CLS (0) and SEP (last)
for i in range(1, len(input_ids) - 1):
    original_id = input_ids[i].item()
    original_token = tokens[i]
    masked_ids = input_ids.clone()
    masked_ids[i] = tokenizer.mask_token_id
    with torch.no_grad():
        logits = model(
            masked_ids.unsqueeze(0),
            attention_mask=attention_mask).logits
    log_probs = torch.log_softmax(logits[0, i], dim=-1)
    log_prob_original = log_probs[original_id]
    ppl = torch.exp(-log_prob_original).item()
    
    ppl_weird.append(ppl)
    save_tokens.append(original_token)


In [ ]:
zipped_results = list(zip(save_tokens, ppl_weird))
np.savetxt('data/twelve_heads_replacement_perplexity_results_500.txt', zipped_results, fmt='%s', delimiter='\t')

In [ ]:
for token, ppl in zip(save_tokens, ppl_weird):
    print(f"Token: {token}\t\tPerplexity: {ppl}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Setup the sample data (recreating the DataFrame)
# The data provided by the user:

sentence = full_story

# Encode with special tokens
enc = tokenizer(sentence, return_tensors="pt", add_special_tokens=True)
input_ids = enc["input_ids"][0]
attention_mask = enc["attention_mask"]

# Convert ids → tokens (these include [CLS] and [SEP])
tokens = tokenizer.convert_ids_to_tokens(input_ids)

per_token_ppl = []

# Loop over tokens EXCLUDING CLS (0) and SEP (last)
for i in range(1, len(input_ids) - 1):

    original_id = input_ids[i].item()
    original_token = tokens[i]

    masked_ids = input_ids.clone()
    masked_ids[i] = tokenizer.mask_token_id

    with torch.no_grad():
        logits = model(
            masked_ids.unsqueeze(0),
            attention_mask=attention_mask
        ).logits

    log_probs = torch.log_softmax(logits[0, i], dim=-1)
    log_prob_original = log_probs[original_id]

    ppl = torch.exp(-log_prob_original).item()

    per_token_ppl.append((original_token, ppl))

tokens, perplexities = zip(*per_token_ppl)
    
raw_data = np.array([
    [token, f"{prob:.2f}"] for token, prob in zip(tokens, perplexities)
])

# Create a DataFrame for context (assuming columns were 'Token' and 'Perplexity')
df = pd.DataFrame(raw_data, columns=['Token', 'Perplexity'])

# Convert Perplexity column to numeric for calculations
perplexity_values = df['Perplexity'].astype(float)

# 2. Setup the plot and transpose the table data
fig, ax = plt.subplots(figsize=(16, 1.25)) # Slightly increased height for statistics text
ax.axis('off') # Hide the axis lines

# --- Core modification: Transposing the data ---
cell_text = df.values.T

# Define the new row labels based on the original columns
row_labels = ['Token', 'Perplexity']

# 3. Create the table with transposed data and updated labels
# --- Modification 1: Setting colLabels=None to remove the empty header row ---
table = ax.table(
    cellText=cell_text,
    rowLabels=row_labels,  # 'Token' and 'Perplexity' are now row headers
    colLabels=None,        # Removed column labels to eliminate the top empty row
    cellLoc='center',
    loc='center')

# Optional: Adjust table properties
table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1, 2) # Scale height slightly for better spacing

# 4. Calculate and display statistics below the table
median_val = np.median(perplexity_values)
mean_val = np.mean(perplexity_values)
std_val = np.std(perplexity_values)

stats_text = (
    f"Perplexity Statistics | MODEL UNTOUCHED:\n"
    f"Median: {median_val:.2f} | Mean: {mean_val:.2f} | Std Dev: {std_val:.0f}"
)

# --- Modification 2: Removed plt.title() ---
plt.title(stats_text)
plt.show()

In [ ]:
raw_data

In [ ]:
perplexity_values

In [ ]:
df.values

In [ ]:
print('\nWeird: ', np.median(ppl_zero_30), '\t', np.mean(ppl_zero_30))
print('Normal: ', np.median(ppl_subset_normal), '\t', np.mean(ppl_subset_normal), '\n')

print(f"Average Perplexity weird: pd-describe{(pd.Series(ppl_subset_replaced).describe())}\n")
print(f"Average Perplexity normal: pd-describe{(pd.Series(ppl_subset_normal).describe())}")